<a id='Top'></a>
# 22. Inheritance
<div class="alert alert-block alert-danger" style="margin-top: 20px">
<font color=black>

- 22.1. [Introduction: Class Inheritance](#22.1)
- 22.2. [Inheriting Variables and Methods](#22.2)
  - 22.2.1. [Mechanics of Defining a Subclass](#22.2.1)
  - 22.2.2. [How the interpreter looks up attributes](#22.2.2)
- 22.3. [Overriding Methods](#22.3)
- 22.4. [Invoking the Parent Class’s Method](#22.4)
- 22.5. [Tamagotchi Revisited](#22.5)
- 22.6. [Exercises](#22.6)
- 22.7. [Chapter Assessment](#22.7)
- 22.8. [Project - Wheel of Python](#22.8)</div>

<a id='22.1'></a>
## 22.1. Introduction: Class Inheritance
Classes can “inherit” methods and class variables from other classes. We’ll see the mechanics of how this works in subsequent sections. First, however, let’s motivate why this might be valuable. It turns out that inheritance doesn’t let you do anything that you couldn’t do without it, but it makes some things a lot more elegant. You will also find it’s useful when someone else has defined a class in a module or library, and you just want to override a few things without having to reimplement everything they’ve done.

Consider our Tamagotchi game. Suppose we wanted to make some different kinds of pets that have the same structure as other pets, but have some different attributes or behave a little differently. For example, suppose that dog pets should show their emotional state a little differently than cats or act differently when they are hungry or when they are asked to fetch something.

You could implement this by making an instance variable for the pet type and dispatching on that instance variable in various methods.

In [ ]:
from random import randrange

class Pet():
    boredom_decrement = 4
    hunger_decrement = 6
    boredom_threshold = 5
    hunger_threshold = 10
    sounds = ['Mrrp']
    def __init__(self, name = "Kitty", pet_type="dog"):
        self.name = name
        self.hunger = randrange(self.hunger_threshold)
        self.boredom = randrange(self.boredom_threshold)
        self.sounds = self.sounds[:]  # copy the class attribute, so that when we make changes to it, we won't affect the other Pets in the class
        self.pet_type = pet_type

    def clock_tick(self):
        self.boredom += 1
        self.hunger += 1

    def mood(self):
        if self.hunger <= self.hunger_threshold and self.boredom <= self.boredom_threshold:
            if self.pet_type == "dog": # if the pet is a dog, it will express its mood in different ways from a cat or any other type of animal
                return "happy"
            elif self.pet_type == "cat":
                return "happy, probably"
            else:
                return "HAPPY"
        elif self.hunger > self.hunger_threshold:
            if self.pet_type == "dog": # same for hunger -- dogs and cats will express their hunger a little bit differently in this version of the class definition
                return "hungry, arf"
            elif self.pet_type == "cat":
                return "hungry, meeeeow"
            else:
                return "hungry"
        else:
            return "bored"

    def __str__(self):
        state = "     I'm " + self.name + ". "
        state += " I feel " + self.mood() + ". "
        return state

    def hi(self):
        print(self.sounds[randrange(len(self.sounds))])
        self.reduce_boredom()

    def teach(self, word):
        self.sounds.append(word)
        self.reduce_boredom()

    def feed(self):
        self.reduce_hunger()

    def reduce_hunger(self):
        self.hunger = max(0, self.hunger - self.hunger_decrement)

    def reduce_boredom(self):
        self.boredom = max(0, self.boredom - self.boredom_decrement)

**That code is exactly the same as the code defining the <font color=red>Pet</font> class that you saw in the <font color=blue>Tamagotchi</font> section, except that we’ve added a few things.**
  * A new input to the constructor – the <font color=red>pet_type</font> input parameter, which defaults to <font color=red>"dog"</font>, and the  
  <font color=red>self.pet_type</font> instance variable.
  * if..elif..else in the <font color=red>self.mood()</font> method, such that different types of pets (a dog, a cat, or any other  
  type of animal) express their moods and their hunger in slightly different ways.

But that’s not an elegant way to do it. It obscures the parts of being a pet that are common to all pets and it buries the unique stuff about being a dog or a cat in the middle of the mood method. What if you also wanted a dog to reduce boredom at a different rate than a cat, and you wanted a bird pet to be different still? Here, we’ve only implemented __dogs__, __cats__, and __other__ – but you can imagine the possibilities.

If there were lots of different types of pets, those methods would start to have long and complex __if..elif..elif__ code clauses, which can be confusing. And you’d need that in every method where the behavior was different for different types of pets. Class inheritance will give us a more elegant way to do it.

<a id='22.2'></a>
## 22.2. Inheriting Variables and Methods
<a id='22.2.1'></a>
### 22.2.1. Mechanics of Defining a Subclass
[Back to top](#Top)

We said that inheritance provides us a more elegant way of, for example, creating <font color=red>Dog</font> and <font color=red>Cat</font> types, rather than making a very complex <font color=red>Pet</font> class. In the abstract, this is pretty intuitive: all pets have certain things, but dogs are different from cats, which are different from birds. Going a step further, a Collie dog is different from a Labrador dog, for example. Inheritance provides us with an easy and elegant way to represent these differences.

Basically, it works by defining a new class, and using a special syntax to show what the new sub-class inherits from a super-class. So if you wanted to define a <font color=red>Dog</font> class as a special kind of <font color=red>Pet</font>, you would say that the <font color=red>Dog</font> type inherits from the <font color=red>Pet</font> type. In the definition of the inherited class, you only need to specify the methods and instance variables that are different from the parent class (the parent class, or the superclass, is what we may call the class that is inherited from. In the example we’re discussing, <font color=red>Pet</font> would be the superclass of <font color=red>Dog</font> or <font color=red>Cat</font>).

Here is an example. Say we want to define a class <font color=red>Cat</font> that inherits from <font color=red>Pet</font>. Assume we have the <font color=red>Pet</font> class that we defined earlier.

We want the <font color=red>Cat</font> type to be exactly the same as <font color=red>Pet</font>, except we want the sound cats to start out knowing “meow” instead of “mrrp”, and we want the <font color=red>Cat</font> class to have its own special method called <font color=red>chasing_rats</font>, which only <font color=red>Cat</font> s have.

For reference, here’s the original Tamagotchi code

In [ ]:
from random import randrange

# Here's the original Pet class
class Pet():
    boredom_decrement = 4
    hunger_decrement = 6
    boredom_threshold = 5
    hunger_threshold = 10
    sounds = ['Mrrp']
    def __init__(self, name = "Kitty"):
        self.name = name
        self.hunger = randrange(self.hunger_threshold)
        self.boredom = randrange(self.boredom_threshold)
        self.sounds = self.sounds[:]  # copy the class attribute, so that when we make changes to it, we won't affect the other Pets in the class

    def clock_tick(self):
        self.boredom += 1
        self.hunger += 1

    def mood(self):
        if self.hunger <= self.hunger_threshold and self.boredom <= self.boredom_threshold:
            return "happy"
        elif self.hunger > self.hunger_threshold:
            return "hungry"
        else:
            return "bored"

    def __str__(self):
        state = "     I'm " + self.name + ". "
        state += " I feel " + self.mood() + ". "
        # state += "Hunger %d Boredom %d Words %s" % (self.hunger, self.boredom, self.sounds)
        return state

    def hi(self):
        print(self.sounds[randrange(len(self.sounds))])
        self.reduce_boredom()

    def teach(self, word):
        self.sounds.append(word)
        self.reduce_boredom()

    def feed(self):
        self.reduce_hunger()

    def reduce_hunger(self):
        self.hunger = max(0, self.hunger - self.hunger_decrement)

    def reduce_boredom(self):
        self.boredom = max(0, self.boredom - self.boredom_decrement)

# Here's the new definition of class Cat, a subclass of Pet.
class Cat(Pet): # the class name that the new class inherits from goes in the parentheses, like so.
    sounds = ['Meow']

    def chasing_rats(self):
        return "What are you doing, Pinky? Taking over the world?!"

All we need is the few extra lines at the bottom of the ActiveCode window! The elegance of inheritance allows us to specify just the differences in the new, inherited class. In that extra code, we make sure the <font color=red>Cat</font> class inherits from the <font color=red>Pet</font> class. We do that by putting the word Pet in parentheses, <font color=red>class Cat(Pet):</font>. In the definition of the class <font color=red>Cat</font>, we only need to define the things that are different from the ones in the <font color=red>Pet</font> class.

In this case, the only difference is that the class variable <font color=red>sounds</font> starts out with the string <font color=red>"Meow"</font> instead of the string <font color=red>"mrrp"</font>, and there is a new method <font color=red>chasing_rats</font>.

We can still use all the <font color=red>Pet</font> methods in the <font color=red>Cat</font> class, this way. You can call the <font color=red>\_\_str\_\_</font> method on an instance of <font color=red>Cat</font> to <font color=red>print</font> an instance of <font color=red>Cat</font>, the same way you could call it on an instance of <font color=red>Pet</font>, and the same is true for the <font color=red>hi</font> method – it’s the same for instances of <font color=red>Cat</font> and <font color=red>Pet</font>. But the <font color=red>chasing_rats</font> method is special: it’s only usable on <font color=red>Cat</font> instances, because <font color=red>Cat</font> is a subclass of <font color=red>Pet</font> which has that additional method.

In the original Tamagotchi game in the last chapter, you saw code that created instances of the <font color=red>Pet</font> class. Now let’s write a little bit of code that uses instances of the <font color=red>Pet</font> class AND instances of the <font color=red>Cat</font> class.

In [ ]:
p1 = Pet("Fido")
print(p1) # we've seen this stuff before!

p1.feed()
p1.hi()
print(p1)

cat1 = Cat("Fluffy")
print(cat1) # this uses the same __str__ method as the Pets do

cat1.feed() # Totally fine, because the cat class inherits from the Pet class!
cat1.hi()
print(cat1)

print(cat1.chasing_rats())

#print(p1.chasing_rats()) # This line will give us an error. The Pet class doesn't have this method!

And you can continue the inheritance tree. We inherited <font color=red>Cat</font> from <font color=red>Pet</font>. Now say we want a subclass of <font color=red>Cat</font> called <font color=red>Cheshire</font>. A Cheshire cat should inherit everything from <font color=red>Cat</font>, which means it inherits everything that <font color=red>Cat</font> inherits from <font color=red>Pet</font>, too. But the <font color=red>Cheshire</font> class has its own special method, <font color=red>smile</font>.

In [ ]:
class Cheshire(Cat): # this inherits from Cat, which inherits from Pet

    def smile(self): # this method is specific to instances of Cheshire
        print(":D :D :D")

# Let's try it with instances.
cat1 = Cat("Fluffy")
cat1.feed() # Totally fine, because the cat class inherits from the Pet class!
cat1.hi() # Uses the special Cat hello.
print(cat1)

print(cat1.chasing_rats())

new_cat = Cheshire("Pumpkin") # create a Cheshire cat instance with name "Pumpkin"
new_cat.hi() # same as Cat!
new_cat.chasing_rats() # OK, because Cheshire inherits from Cat
new_cat.smile() # Only for Cheshire instances (and any classes that you make inherit from Cheshire)

# cat1.smile() # This line would give you an error, because the Cat class does not have this method!

# None of the subclass methods can be used on the parent class, though.
p1 = Pet("Teddy")
p1.hi() # just the regular Pet hello
#p1.chasing_rats() # This will give you an error -- this method doesn't exist on instances of the Pet class.
#p1.smile() # This will give you an error, too. This method does not exist on instances of the Pet class.

<a id='22.2.2'></a>
### 22.2.2. How the interpreter looks up attributes
[Back to top](#Top)

So what is happening in the Python interpreter when you write programs with classes, subclasses, and instances of both parent classes and subclasses?

**This is how the interpreter looks up attributes:**

1. First, it checks for an instance variable or an instance method by the name it’s looking for.
2. If an instance variable or method by that name is not found, it checks for a class variable. (See the <font color=blue>previous chapter</font> for an explanation of the difference between __instance variables__ and __class variables__.)
3. If no class variable is found, it looks for a class variable in the parent class.
4. If no class variable is found, the interpreter looks for a class variable in THAT class’s parent (the “grandparent” class).
5. This process goes on until the last ancestor is reached, at which point Python will signal an error.

Let’s look at this with respect to some code.

Say you write the lines:

In [ ]:
new_cat = Cheshire("Pumpkin")
print(new_cat.name)

In the second line, after the instance is created, Python looks for the instance variable <font color=red>name</font> in the <font color=red>new_cat</font> instance. In this case, it exists. The name on this instance of <font color=red>Cheshire</font> is <font color=red>Pumpkin</font>. There you go!

When the following lines of code are written and executed:

In [ ]:
cat1 = Cat("Sepia")
cat1.hi()

The Python interpreter looks for <font color=red>hi</font> in the instance of <font color=red>Cat</font>. It does not find it, because there’s no statement of the form <font color=red>cat1.hi = ...</font>. (Be careful here – if you had set an instance variable on Cat called <font color=red>hi</font> it would be a bad idea, because you would not be able to use the __method__ that it inherited anymore. We’ll see more about this later.)

Then it looks for hi as a class variable (or method) in the class Cat, and still doesn’t find it.

Next, it looks for a class variable <font color=red>hi</font> on the parent class of <font color=red>Cat</font>, <font color=red>Pet</font>. It finds that – there’s a method called <font color=red>hi</font> on the class <font color=red>Pet</font>. Because of the <font color=red>()</font> after <font color=red>hi</font>, the method is invoked. All is well.

However, for the following, it won’t go so well

In [ ]:
p1 = Pet("Teddy")
p1.chasing_rats()

The Python interpreter looks for an instance variable or method called <font color=red>chasing_rats</font> on the <font color=red>Pet</font> class. It doesn’t exist. <font color=red>Pet</font> has no parent classes, so Python signals an error.

#### Check your understanding
<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>

1. After you run the code, <font color=red>new_cat = Cheshire("Pumpkin")</font>, how many instance variables exist for the new_cat instance of Cheshire?

  A. 1  
  B. 2  
  C. 3  
  D. 4  


<details><summary>Click here for the solution</summary>

<font color=red>► </font>D. 4

<div class="alert alert-block alert-success" style="margin-top: 20px">
<font color=black>✔️ Neither Cheshire nor Cat defines an __init__ constructor method, so the grandaprent class, Pet, will have it's __init__ method called. That constructor method sets the instance variables name, hunger, boredom, and sounds.

</details>

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>

2. What would print after running the following code:

In [ ]:
new_cat = Cheshire("Pumpkin”)
class Siamese(Cat):
  def song(self):
    print("We are Siamese if you please. We are Siamese if you don’t please.")
another_cat = Siamese("Lady")
another_cat.song()

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>

A. We are Siamese if you please. We are Siamese if you don’t please.    
B. Error  
C. Pumpkin  
D. Nothing. There’s no print statement.
    

<details><summary>Click here for the solution</summary>

<font color=red>► </font>A. We are Siamese if you please. We are Siamese if you don’t please.

<div class="alert alert-block alert-success" style="margin-top: 20px">
<font color=black>✔️ another_cat is an instance of Siamese, so its song() method is invoked.

</details>

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>

3. What would print after running the following code:

In [ ]:
new_cat = Cheshire("Pumpkin”)
class Siamese(Cat):
  def song(self):
    print("We are Siamese if you please. We are Siamese if you don’t please.")
another_cat = Siamese("Lady")
new_cat.song()

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>

A. We are Siamese if you please. We are Siamese if you don’t please.  
B. Error    
C. Pumpkin  
D. Nothing. There’s no print statement.

<details><summary>Click here for the solution</summary>

<font color=red>► </font>B. Error

<div class="alert alert-block alert-success" style="margin-top: 20px">
<font color=black>✔️ You cannot invoke methods defined in the Siamese class on an instance of the Cheshire class. Both are subclasses of Cat, but Cheshire is not a subclass of Siamese, so it doesn't inherit its methods.

</details>

<a id='22.3'></a>
## 22.3. Overriding Methods
[Back to top](#Top)

If a method is defined for a class, and also defined for its parent class, the subclass’ method is called and not the parent’s. This follows from the rules for looking up attributes that you saw in the previous section.

We can use the same idea to understand overriding methods.

Let’s return to our idea of making Cats, Dogs, and other pets generate a string for their “mood” differently.

Here’s the original Pet class again.

In [ ]:
from random import randrange

# Here's the original Pet class
class Pet():
    boredom_decrement = 4
    hunger_decrement = 6
    boredom_threshold = 5
    hunger_threshold = 10
    sounds = ['Mrrp']
    def __init__(self, name = "Kitty"):
        self.name = name
        self.hunger = randrange(self.hunger_threshold)
        self.boredom = randrange(self.boredom_threshold)
        self.sounds = self.sounds[:]  # copy the class attribute, so that when we make changes to it, we won't affect the other Pets in the class

    def clock_tick(self):
        self.boredom += 1
        self.hunger += 1

    def mood(self):
        if self.hunger <= self.hunger_threshold and self.boredom <= self.boredom_threshold:
            return "happy"
        elif self.hunger > self.hunger_threshold:
            return "hungry"
        else:
            return "bored"

    def __str__(self):
        state = "     I'm " + self.name + ". "
        state += " I feel " + self.mood() + ". "
        # state += "Hunger %d Boredom %d Words %s" % (self.hunger, self.boredom, self.sounds)
        return state

    def hi(self):
        print(self.sounds[randrange(len(self.sounds))])
        self.reduce_boredom()

    def teach(self, word):
        self.sounds.append(word)
        self.reduce_boredom()

    def feed(self):
        self.reduce_hunger()

    def reduce_hunger(self):
        self.hunger = max(0, self.hunger - self.hunger_decrement)

    def reduce_boredom(self):
        self.boredom = max(0, self.boredom - self.boredom_decrement)

Now let’s make two subclasses, Dog and Cat. Dogs are always happy unless they are bored and hungry. Cats, on the other hand, are happy only if they are fed and if their boredom level is in a narrow range and, even then, only with probability 1/2.

In [ ]:
class Cat(Pet):
    sounds = ['Meow']

    def mood(self):
        if self.hunger > self.hunger_threshold:
            return "hungry"
        if self.boredom <2:
            return "grumpy; leave me alone"
        elif self.boredom > self.boredom_threshold:
            return "bored"
        elif randrange(2) == 0:
            return "randomly annoyed"
        else:
            return "happy"

class Dog(Pet):
    sounds = ['Woof', 'Ruff']

    def mood(self):
        if (self.hunger > self.hunger_threshold) and (self.boredom > self.boredom_threshold):
            return "bored and hungry"
        else:
            return "happy"

c1 = Cat("Fluffy")
d1 = Dog("Astro")

c1.boredom = 1
print(c1.mood())
c1.boredom = 3
for i in range(10):
    print(c1.mood())
print(d1.mood())

<a id='22.4'></a>
## 22.4. Invoking the Parent Class’s Method
[Back to top](#Top)

Sometimes the parent class has a useful method, but you just need to execute a little extra code when running the subclass’s method. You can override the parent class’s method in the subclass’s method with the same name, but also invoke the parent class’s method. Here’s how.

Say you wanted the <font color=red>Dog</font> subclass of <font color=red>Pet</font> to say “Arf! Thanks!” when the <font color=red>feed</font> method is called, as well as executing the code in the original method.

Here’s the original Pet class again.

In [ ]:
from random import randrange

# Here's the original Pet class
class Pet():
    boredom_decrement = 4
    hunger_decrement = 6
    boredom_threshold = 5
    hunger_threshold = 10
    sounds = ['Mrrp']
    def __init__(self, name = "Kitty"):
        self.name = name
        self.hunger = randrange(self.hunger_threshold)
        self.boredom = randrange(self.boredom_threshold)
        self.sounds = self.sounds[:]  # copy the class attribute, so that when we make changes to it, we won't affect the other Pets in the class

    def clock_tick(self):
        self.boredom += 1
        self.hunger += 1

    def mood(self):
        if self.hunger <= self.hunger_threshold and self.boredom <= self.boredom_threshold:
            return "happy"
        elif self.hunger > self.hunger_threshold:
            return "hungry"
        else:
            return "bored"

    def __str__(self):
        state = "     I'm " + self.name + ". "
        state += " I feel " + self.mood() + ". "
        # state += "Hunger %d Boredom %d Words %s" % (self.hunger, self.boredom, self.sounds)
        return state

    def hi(self):
        print(self.sounds[randrange(len(self.sounds))])
        self.reduce_boredom()

    def teach(self, word):
        self.sounds.append(word)
        self.reduce_boredom()

    def feed(self):
        self.reduce_hunger()

    def reduce_hunger(self):
        self.hunger = max(0, self.hunger - self.hunger_decrement)

    def reduce_boredom(self):
        self.boredom = max(0, self.boredom - self.boredom_decrement)

And here’s a subclass that overrides feed() by invoking the the parent class’s feed() method; it then also executes an extra line of code. Note the somewhat inelegant way of invoking the parent class’ method. We explicitly refer to Pet.feed to get the method/function object. We invoke it with parentheses. However, since we are not invoking the method the normal way, with \<obj\>.methodname, we have to explicitly pass an instance as the first parameter. In this case, the variable self in Dog.feed() will be bound to an instance of Dog, and so we can just pass self: <font color=red>Pet.feed(self)</font>.

In [ ]:
from random import randrange

class Dog(Pet):
    sounds = ['Woof', 'Ruff']

    def feed(self):
        Pet.feed(self)
        print("Arf! Thanks!")

d1 = Dog("Astro")
d1.feed()

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <font color=black><b>Note</b><br>  
There’s a better way to invoke a superclass’s method. Unfortunately, the implementation of python in our ActiveCode windows doesn’t support it, so we aren’t using it here. In that alternative method, we would call <font color=red>super().feed()</font>. This is nice because it’s easier to read, and also because it puts the specification of the class that Dog inherits from in just one place, <font color=red>class Dog(Pet)</font>. Elsewhere, you just refer to <font color=red>super()</font> and python takes care of looking up that the parent (super) class of Dog is Pet.</div>

This technique is very often used with the <font color=red>\_\_init\_\_</font> method for a subclass. Suppose that some extra instance variables are defined for the subclass. When you invoke the constructor, you pass all the regular parameters for the parent class, plus the extra ones for the subclass. The subclass’ <font color=red>\_\_init\_\_</font> method then stores the extra parameters in instance variables and calls the parent class’ <font color=red>\_\_init\_\_</font> method to store the common parameters in instance variables and do any other initialization that it normally does.

Let’s say we want to create a subclass of <font color=red>Pet</font>, called <font color=red>Bird</font>, and we want it to take an extra parameter, <font color=red>chirp_number</font>, with a default value of 2, and have an extra instance variable, <font color=red>self.chirp_number</font>. Then, we’ll use this in the <font color=red>hi</font> method to make more than one sound.

In [ ]:
class Bird(Pet):
    sounds = ["chirp"]
    def __init__(self, name="Kitty", chirp_number=2):
        Pet.__init__(self, name) # call the parent class's constructor
        # basically, call the SUPER -- the parent version -- of the constructor, with all the parameters that it needs.
        self.chirp_number = chirp_number # now, also assign the new instance variable

    def hi(self):
        for i in range(self.chirp_number):
            print(self.sounds[randrange(len(self.sounds))])
        self.reduce_boredom()

b1 = Bird('tweety', 5)
b1.teach("Polly wanna cracker")
b1.hi()

#### Check your understanding
<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>

1. What will print when <font color=red>print(b1.sounds)</font> is run?

  A. 5  
  B. ["Mrrp"]  
  C. ["chirp"]    
  D. Error


<details><summary>Click here for the solution</summary>

<font color=red>► </font>C. ["chirp"]

<div class="alert alert-block alert-success" style="margin-top: 20px">
<font color=black>✔️ The interpeter finds the value in the class variable for the class Bird.

</details>

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>

2. For the Dog class defined in the earlier activecode window, what would happen when d1.feed() is run if the Pet.feed(self) line was deleted?

  A. Error when invoked  
  B. The string would not print out but d1 would have its hunger reduced.  
  C. The string would print but d1 would not have its hunger reduced.    
  D. Nothing would be different. It is the same as the current code.


<details><summary>Click here for the solution</summary>

<font color=red>► </font>C. The string would print but d1 would not have its hunger reduced.

<div class="alert alert-block alert-success" style="margin-top: 20px">
<font color=black>✔️ Since we are no longer calling the parent Pet class's method in the Dog subclass's method definition, the class definition will override the parent method.

</details>

<a id='22.5'></a>
## 22.5. Tamagotchi Revisited
[Back to top](#Top)

Using what we know about class inheritance, we can make a new version of the Tamagotchi game, where you can adopt different types of pets that are slightly different from one another.

And now we can play the Tamagotchi game with some small changes, such that we can adopt different types of pets.

In [ ]:
import sys
sys.setExecutionLimit(60000)
from random import randrange

class Pet(object):
    boredom_decrement = 4
    hunger_decrement = 6
    boredom_threshold = 5
    hunger_threshold = 10
    sounds = ['Mrrp']
    def __init__(self, name = "Kitty"):
        self.name = name
        self.hunger = randrange(self.hunger_threshold)
        self.boredom = randrange(self.boredom_threshold)
        self.sounds = self.sounds[:]  # copy the class attribute, so that when we make changes to it, we won't affect the other Pets in the class

    def clock_tick(self):
        self.boredom += 1
        self.hunger += 1

    def mood(self):
        if self.hunger <= self.hunger_threshold and self.boredom <= self.boredom_threshold:
            return "happy"
        elif self.hunger > self.hunger_threshold:
            return "hungry"
        else:
            return "bored"

    def __str__(self):
        state = "     I'm " + self.name + ". "
        state += " I feel " + self.mood() + ". "
        # state += "Hunger %d Boredom %d Words %s" % (self.hunger, self.boredom, self.sounds)
        return state

    def hi(self):
        print(self.sounds[randrange(len(self.sounds))])
        self.update_boredom()

    def teach(self, word):
        self.sounds.append(word)
        self.update_boredom()

    def feed(self):
        self.update_hunger()

    def update_hunger(self):
        self.hunger = max(0, self.hunger - self.hunger_decrement)

    def update_boredom(self):
        self.boredom = max(0, self.boredom - self.boredom_decrement)

class Cat(Pet):
    sounds = ['Meow']

    def mood(self):
        if self.hunger > self.hunger_threshold:
            return "hungry"
        if self.boredom <2:
            return "grumpy; leave me alone"
        elif self.boredom > self.boredom_threshold:
            return "bored"
        elif randrange(2) == 0:
            return "randomly annoyed"
        else:
            return "happy"

class Dog(Pet):
    sounds = ['Woof', 'Ruff']

    def mood(self):
        if (self.hunger > self.hunger_threshold) and (self.boredom > self.boredom_threshold):
            return "bored and hungry"
        else:
            return "happy"

    def feed(self):
        Pet.feed(self)
        print("Arf! Thanks!")

class Bird(Pet):
    sounds = ["chirp"]
    def __init__(self, name="Kitty", chirp_number=2):
        Pet.__init__(self, name) # call the parent class's constructor
        # basically, call the SUPER -- the parent version -- of the constructor, with all the parameters that it needs.
        self.chirp_number = chirp_number # now, also assign the new instance variable

    def hi(self):
        for i in range(self.chirp_number):
            print(self.sounds[randrange(len(self.sounds))])
        self.update_boredom()

class Lab(Dog):
    def fetch(self):
        return "I found the tennis ball!"

    def hi(self):
        print(self.fetch())
        print(self.sounds[randrange(len(self.sounds))])

class Poodle(Dog):
    def dance(self):
        return "Dancin' in circles like poodles do."

    def hi(self):
        print(self.dance())
        Dog.hi(self)

def whichone(petlist, name):
    for pet in petlist:
        if pet.name == name:
            return pet
    return None # no pet matched

pet_types = {'dog': Dog, 'lab': Lab, 'poodle': Poodle, 'cat': Cat, 'bird': Bird}
def whichtype(adopt_type="general pet"):
    return pet_types.get(adopt_type.lower(), Pet)

def play():
    animals = []

    option = ""
    base_prompt = """
        Quit
        Adopt <petname_with_no_spaces> <pet_type - choose dog, cat, lab, poodle, bird, or another unknown pet type>
        Greet <petname>
        Teach <petname> <word>
        Feed <petname>

        Choice: """
    feedback = ""
    while True:
        action = input(feedback + "\n" + base_prompt)
        feedback = ""
        words = action.split()
        if len(words) > 0:
            command = words[0]
        else:
            command = None
        if command == "Quit":
            print("Exiting...")
            return
        elif command == "Adopt" and len(words) > 1:
            if whichone(animals, words[1]):
                feedback += "You already have a pet with that name\n"
            else:
                # figure out which class it should be
                if len(words) > 2:
                    Cl = whichtype(words[2])
                else:
                    Cl = Pet
                # Make an instance of that class and append it
                animals.append(Cl(words[1]))
        elif command == "Greet" and len(words) > 1:
            pet = whichone(animals, words[1])
            if not pet:
                feedback += "I didn't recognize that pet name. Please try again.\n"
                print()
            else:
                pet.hi()
        elif command == "Teach" and len(words) > 2:
            pet = whichone(animals, words[1])
            if not pet:
                feedback += "I didn't recognize that pet name. Please try again."
            else:
                pet.teach(words[2])
        elif command == "Feed" and len(words) > 1:
            pet = whichone(animals, words[1])
            if not pet:
                feedback += "I didn't recognize that pet name. Please try again."
            else:
                pet.feed()
        else:
            feedback+= "I didn't understand that. Please try again."

        for pet in animals:
            pet.clock_tick()
            feedback += "\n" + pet.__str__()

play()

<a id='22.6'></a>
## 22.6. Exercises
[Back to top](#Top)

For exercises, you can expand the Tamagotchi game even further. Try these out.

Here’s all the code we just saw for our new and improved game, with a few additions. You can run this and play the game again.
<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>
    
1. Change the code below to allow you to adopt a Tiger pet (that you’re about to create). HINT: look at the <font color=red>whichtype</font> function, and think about what’s happening in the code for that function.
2. Now, modify the code to define a new class, <font color=red>Tiger</font>. The <font color=red>Tiger</font> class should inherit from the <font color=red>Cat</font> class, but its default meow count should be <font color=red>5</font>, not <font color=red>3</font>, and it should have an extra instance method, <font color=red>roar</font>, that prints out the string <font color=red>ROOOOOAR!</font>.
3. Next, modify the code so that when the <font color=red>hi</font> method is called for the <font color=red>Tiger</font> class, the <font color=red>roar</font> method is called. HINT: You’ll have to call one instance method inside another, and you’ll have to redefine a method for the <font color=red>Tiger</font> class. See the  
__overriding methods__ section.
4. Now, modify the code to define another new class, <font color=red>Retriever</font>. This class should inherit from <font color=red>Lab</font>. It should be exactly like <font color=red>Lab</font>, except instead of printing just <font color=red>I found the tennis ball!</font> when the <font color=red>fetch</font> method is called, it should say <font color=red>I found the tennis ball! I can fetch anything!</font>. 
5. Add your own new pets and modifications as you like – remember, to use them in the game, you’ll also have to alter the <font color=red>whichtype</font> function so they can be used in game play. Otherwise, you’ll have different classes that may work just fine, but you won’t see the effects in the game, since the code that actually makes the game play is found in the second half of the provided code (look for the <font color=red>while</font> loop!).

In [ ]:
from random import randrange

class Pet():
    boredom_decrement = 4
    hunger_decrement = 6
    boredom_threshold = 5
    hunger_threshold = 10
    sounds = ['Mrrp']

    def __init__(self, name = "Kitty"):
        self.name = name
        self.hunger = randrange(self.hunger_threshold)
        self.boredom = randrange(self.boredom_threshold)
        self.sounds = self.sounds[:]  # copy the class attribute, so that when we make changes to it, we won't affect the other Pets in the class

    def clock_tick(self):
        self.boredom += 1
        self.hunger += 1

    def mood(self):
        if self.hunger <= self.hunger_threshold and self.boredom <= self.boredom_threshold:
            return "happy"
        elif self.hunger > self.hunger_threshold:
            return "hungry"
        else:
            return "bored"

    def __str__(self):
        state = "     I'm " + self.name + ". "
        state += " I feel " + self.mood() + ". "
        # state += "Hunger %d Boredom %d Words %s" % (self.hunger, self.boredom, self.sounds)
        return state

    def hi(self):
        print(self.sounds[randrange(len(self.sounds))])
        self.reduce_boredom()

    def teach(self, word):
        self.sounds.append(word)
        self.reduce_boredom()

    def feed(self):
        self.reduce_hunger()

    def reduce_hunger(self):
        self.hunger = max(0, self.hunger - self.hunger_decrement)

    def reduce_boredom(self):
        self.boredom = max(0, self.boredom - self.boredom_decrement)

class Dog(Pet):
    # in the Dog class, Dog pets should express their hunger and boredom differently than generic Pets
    def mood(self):
        if self.hunger <= self.hunger_threshold and self.boredom <= self.boredom_threshold:
            return "happy, arf! Happy"
        elif self.hunger > self.hunger_threshold:
            return "hungry already, arrrf"
        else:
            return "bored, so you should play with me"

class Cat(Pet):
    # in the Cat class, cats express their hunger and boredom a little differently, too. They also have an extra instance, variable meow_count.
    def __init__(self, name="Fluffy", meow_count=3):
        Pet.__init__(self, name)
        self.meow_count = meow_count

    def hi(self):
        for i in range(self.meow_count):
            print(self.sounds[randrange(len(self.sounds))])
        self.reduce_boredom()

    def mood(self):
        if self.hunger <= self.hunger_threshold and self.boredom <= self.boredom_threshold:
            return "happy, I suppose"
        elif self.hunger > self.hunger_threshold:
            return "mmmm...hungry"
        else:
            return "a bit bored"

class Lab(Dog):
    def fetch(self):
        return "I found the tennis ball!"

    def hi(self):
        print(self.sounds[randrange(len(self.sounds))] + self.fetch())

class Poodle(Dog):
    def dance(self):
        return "Dancin' in circles like poodles do."

    def hi(self):
        print(self.dance())
        Dog.hi(self)

class Bird(Pet):
    sounds = ["chirp"]
    def __init__(self, name="Kitty", chirp_number=2):
        Pet.__init__(self, name) # call the parent class's constructor
        # basically, call the SUPER -- the parent version -- of the constructor, with all the parameters that it needs.
        self.chirp_number = chirp_number # now, also assign the new instance variable

    def hi(self):
        for i in range(self.chirp_number):
            print(self.sounds[randrange(len(self.sounds))])
        self.reduce_boredom()

def whichone(petlist, name):
    for pet in petlist:
        if pet.name == name:
            return pet
    return None # no pet matched

pet_types = {'dog': Dog, 'lab': Lab, 'poodle': Poodle, 'cat': Cat, 'bird': Bird}
def whichtype(adopt_type="general pet"):
    return pet_types.get(adopt_type.lower(), Pet)

def play():
    animals = []

    option = ""
    base_prompt = """
        Quit
        Adopt <petname_with_no_spaces> <adopt_type - choose dog, cat, lab, poodle, or another unknown pet type>
        Greet <petname>
        Teach <petname> <word>
        Feed <petname>

        Choice: """
    feedback = ""
    while True:
        action = input(feedback + "\n" + base_prompt)
        feedback = ""
        words = action.split()
        if len(words) > 0:
            command = words[0]
        else:
            command = None
        if command == "Quit":
            print("Exiting...")
            return
        elif command == "Adopt" and len(words) > 1:
            if whichone(animals, words[1]):
                feedback += "You already have a pet with that name\n"
            else:
                # figure out which class it should be
                if len(words) > 2:
                    Cl = whichtype(words[2])
                else:
                    Cl = Pet
                # Make an instance of that class and append it
                animals.append(Cl(words[1]))
        elif command == "Greet" and len(words) > 1:
            pet = whichone(animals, words[1])
            if not pet:
                feedback += "I didn't recognize that pet name. Please try again.\n"
                print()
            else:
                pet.hi()
        elif command == "Teach" and len(words) > 2:
            pet = whichone(animals, words[1])
            if not pet:
                feedback += "I didn't recognize that pet name. Please try again."
            else:
                pet.teach(words[2])
        elif command == "Feed" and len(words) > 1:
            pet = whichone(animals, words[1])
            if not pet:
                feedback += "I didn't recognize that pet name. Please try again."
            else:
                pet.feed()
        else:
            feedback+= "I didn't understand that. Please try again."

        for pet in animals:
            pet.clock_tick()
            feedback += "\n" + pet.__str__()

import sys
sys.setExecutionLimit(60000)
play()

<details><summary>Click here for a solution</summary>
<div class="alert alert-block alert-success" style="margin-top: 20px">
<font color=black>
    
```python
from random import randrange

class Pet():
    boredom_decrement = 4
    hunger_decrement = 6
    boredom_threshold = 5
    hunger_threshold = 10
    sounds = ['Mrrp']

    def __init__(self, name = "Kitty"):
        self.name = name
        self.hunger = randrange(self.hunger_threshold)
        self.boredom = randrange(self.boredom_threshold)
        self.sounds = self.sounds[:]  # copy the class attribute, so that when we make changes to it, we won't affect the other Pets in the class

    def clock_tick(self):
        self.boredom += 1
        self.hunger += 1

    def mood(self):
        if self.hunger <= self.hunger_threshold and self.boredom <= self.boredom_threshold:
            return "happy"
        elif self.hunger > self.hunger_threshold:
            return "hungry"
        else:
            return "bored"

    def __str__(self):
        state = "     I'm " + self.name + ". "
        state += " I feel " + self.mood() + ". "
        # state += "Hunger %d Boredom %d Words %s" % (self.hunger, self.boredom, self.sounds)
        return state

    def hi(self):
        print(self.sounds[randrange(len(self.sounds))])
        self.reduce_boredom()

    def teach(self, word):
        self.sounds.append(word)
        self.reduce_boredom()

    def feed(self):
        self.reduce_hunger()

    def reduce_hunger(self):
        self.hunger = max(0, self.hunger - self.hunger_decrement)

    def reduce_boredom(self):
        self.boredom = max(0, self.boredom - self.boredom_decrement)

class Dog(Pet):
    # in the Dog class, Dog pets should express their hunger and boredom differently than generic Pets
    def mood(self):
        if self.hunger <= self.hunger_threshold and self.boredom <= self.boredom_threshold:
            return "happy, arf! Happy"
        elif self.hunger > self.hunger_threshold:
            return "hungry already, arrrf"
        else:
            return "bored, so you should play with me"

class Cat(Pet):
    # in the Cat class, cats express their hunger and boredom a little differently, too. They also have an extra instance, variable meow_count.
    def __init__(self, name="Fluffy", meow_count=3):
        Pet.__init__(self, name)
        self.meow_count = meow_count

    def hi(self):
        for i in range(self.meow_count):
            print(self.sounds[randrange(len(self.sounds))])
        self.reduce_boredom()

    def mood(self):
        if self.hunger <= self.hunger_threshold and self.boredom <= self.boredom_threshold:
            return "happy, I suppose"
        elif self.hunger > self.hunger_threshold:
            return "mmmm...hungry"
        else:
            return "a bit bored"

class Lab(Dog):
    def fetch(self):
        return "I found the tennis ball!"

    def hi(self):
        print(self.sounds[randrange(len(self.sounds))] + self.fetch())

class Poodle(Dog):
    def dance(self):
        return "Dancin' in circles like poodles do."

    def hi(self):
        print(self.dance())
        Dog.hi(self)

class Retriever(Lab):
    def fetch(self):
        return "I found the tennis ball! I can fetch anything!"

class Bird(Pet):
    sounds = ["chirp"]
    def __init__(self, name="Kitty", chirp_number=2):
        Pet.__init__(self, name) # call the parent class's constructor
        # basically, call the SUPER -- the parent version -- of the constructor, with all the parameters that it needs.
        self.chirp_number = chirp_number # now, also assign the new instance variable

    def hi(self):
        for i in range(self.chirp_number):
            print(self.sounds[randrange(len(self.sounds))])
        self.reduce_boredom()

class Tiger(Cat):
    def __init__(self, name):
        Cat.__init__(self, name, meow_count)
        self.meow_count = 5
    
    def hi(self):
        self.roar()
        
    def roar(self):
        return 'ROOOOOAR!'

def whichone(petlist, name):
    for pet in petlist:
        if pet.name == name:
            return pet
    return None # no pet matched

pet_types = {'dog': Dog, 'lab': Lab, 'poodle': Poodle, 'cat': Cat, 'bird': Bird, 'tiger': Tiger}

def whichtype(adopt_type="general pet"):
    return pet_types.get(adopt_type.lower(), Pet)

def play():
    animals = []

    option = ""
    base_prompt = """
        Quit
        Adopt <petname_with_no_spaces> <adopt_type - choose dog, cat, lab, poodle, tiger, or another unknown pet type>
        Greet <petname>
        Teach <petname> <word>
        Feed <petname>

        Choice: """
    feedback = ""
    while True:
        action = input(feedback + "\n" + base_prompt)
        feedback = ""
        words = action.split()
        if len(words) > 0:
            command = words[0]
        else:
            command = None
        if command == "Quit":
            print("Exiting...")
            return
        elif command == "Adopt" and len(words) > 1:
            if whichone(animals, words[1]):
                feedback += "You already have a pet with that name\n"
            else:
                # figure out which class it should be
                if len(words) > 2:
                    Cl = whichtype(words[2])
                else:
                    Cl = Pet
                # Make an instance of that class and append it
                animals.append(Cl(words[1]))
        elif command == "Greet" and len(words) > 1:
            pet = whichone(animals, words[1])
            if not pet:
                feedback += "I didn't recognize that pet name. Please try again.\n"
                print()
            else:
                pet.hi()
        elif command == "Teach" and len(words) > 2:
            pet = whichone(animals, words[1])
            if not pet:
                feedback += "I didn't recognize that pet name. Please try again."
            else:
                pet.teach(words[2])
        elif command == "Feed" and len(words) > 1:
            pet = whichone(animals, words[1])
            if not pet:
                feedback += "I didn't recognize that pet name. Please try again."
            else:
                pet.feed()
        else:
            feedback+= "I didn't understand that. Please try again."

        for pet in animals:
            pet.clock_tick()
            feedback += "\n" + pet.__str__()

import sys
sys.setExecutionLimit(60000)
play()   
```

</details>

<a id='22.7'></a>
## 22.7. Chapter Assessment
[Back to top](#Top)

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>

1. The class, <font color=red>Pokemon</font>, is provided below and describes a Pokemon and its leveling and evolving characteristics. An instance of the class is one pokemon that you create.

  <font color=red>Grass_Pokemon</font> is a subclass that inherits from <font color=red>Pokemon</font> but changes some aspects, for instance, the boost values are different.

  For the subclass <font color=red>Grass_Pokemon</font>, add another method called <font color=red>action</font> that returns the string <font color=red>"[name of pokemon] knows a lot of different moves!"</font>. Create an instance of this class with the <font color=red>name</font> as <font color=red>"Belle"</font>. Assign this instance to the variable <font color=red>p1</font>.

In [ ]:
class Pokemon(object):
    attack = 12
    defense = 10
    health = 15
    p_type = "Normal"

    def __init__(self, name, level = 5):
        self.name = name
        self.level = level

    def train(self):
        self.update()
        self.attack_up()
        self.defense_up()
        self.health_up()
        self.level = self.level + 1
        if self.level%self.evolve == 0:
            return self.level, "Evolved!"
        else:
            return self.level

    def attack_up(self):
        self.attack = self.attack + self.attack_boost
        return self.attack

    def defense_up(self):
        self.defense = self.defense + self.defense_boost
        return self.defense

    def health_up(self):
        self.health = self.health + self.health_boost
        return self.health

    def update(self):
        self.health_boost = 5
        self.attack_boost = 3
        self.defense_boost = 2
        self.evolve = 10

    def __str__(self):
        self.update()
        return "Pokemon name: {}, Type: {}, Level: {}".format(self.name, self.p_type, self.level)

class Grass_Pokemon(Pokemon):
    attack = 15
    defense = 14
    health = 12

    def update(self):
        self.health_boost = 6
        self.attack_boost = 2
        self.defense_boost = 3
        self.evolve = 12

    def moves(self):
        self.p_moves = ["razor leaf", "synthesis", "petal dance"]

<details><summary>Click here for a solution</summary>
<div class="alert alert-block alert-success" style="margin-top: 20px">
<font color=black>
    
```python
class Pokemon(object):
    attack = 12
    defense = 10
    health = 15
    p_type = "Normal"

    def __init__(self, name, level = 5):
        self.name = name
        self.level = level

    def train(self):
        self.update()
        self.attack_up()
        self.defense_up()
        self.health_up()
        self.level = self.level + 1
        if self.level%self.evolve == 0:
            return self.level, "Evolved!"
        else:
            return self.level

    def attack_up(self):
        self.attack = self.attack + self.attack_boost
        return self.attack

    def defense_up(self):
        self.defense = self.defense + self.defense_boost
        return self.defense

    def health_up(self):
        self.health = self.health + self.health_boost
        return self.health

    def update(self):
        self.health_boost = 5
        self.attack_boost = 3
        self.defense_boost = 2
        self.evolve = 10

    def __str__(self):
        self.update()
        return "Pokemon name: {}, Type: {}, Level: {}".format(self.name, self.p_type, self.level)

class Grass_Pokemon(Pokemon):
    attack = 15
    defense = 14
    health = 12

    def update(self):
        self.health_boost = 6
        self.attack_boost = 2
        self.defense_boost = 3
        self.evolve = 12

    def moves(self):
        self.p_moves = ["razor leaf", "synthesis", "petal dance"]   
```

</details>

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>
    
2. Modify the <font color=red>Grass_Pokemon</font> subclass so that the attack strength for <font color=red>Grass_Pokemon</font> instances does not change until they reach level 10. At level 10 and up, their attack strength should increase by the <font color=red>attack_boost</font> amount when they are trained.

  To test, create an instance of the class with the name as <font color=red>"Bulby"</font>. Assign the instance to the variable <font color=red>p2</font>. Create another instance of the <font color=red>Grass_Pokemon</font> class with the name set to <font color=red>"Pika"</font> and assign that instance to the variable <font color=red>p3</font>.. Then, use <font color=red>Grass_Pokemon</font> methods to train the <font color=red>p3 Grass_Pokemon</font> instance until it reaches at least level 10.

In [4]:
class Pokemon(object):
    attack = 12
    defense = 10
    health = 15
    p_type = "Normal"

    def __init__(self, name, level = 5):
        self.name = name
        self.level = level

    def train(self):
        self.update()
        self.attack_up()
        self.defense_up()
        self.health_up()
        self.level = self.level + 1
        if self.level%self.evolve == 0:
            return self.level, "Evolved!"
        else:
            return self.level

    def attack_up(self):
        self.attack = self.attack + self.attack_boost
        return self.attack

    def defense_up(self):
        self.defense = self.defense + self.defense_boost
        return self.defense

    def health_up(self):
        self.health = self.health + self.health_boost
        return self.health

    def update(self):
        self.health_boost = 5
        self.attack_boost = 3
        self.defense_boost = 2
        self.evolve = 10

    def __str__(self):
        return "Pokemon name: {}, Type: {}, Level: {}".format(self.name, self.p_type, self.level)

class Grass_Pokemon(Pokemon):
    attack = 15
    defense = 14
    health = 12
    p_type = "Grass"

    def update(self):
        self.health_boost = 6
        self.attack_boost = 2
        self.defense_boost = 3
        self.evolve = 12

    def moves(self):
        self.p_moves = ["razor leaf", "synthesis", "petal dance"]

6

<details><summary>Click here for a solution</summary>
<div class="alert alert-block alert-success" style="margin-top: 20px">
<font color=black>
    
```python
class Pokemon(object):
    attack = 12
    defense = 10
    health = 15
    p_type = "Normal"

    def __init__(self, name, level = 5):
        self.name = name
        self.level = level

    def train(self):
        self.update()
        self.attack_up()
        self.defense_up()
        self.health_up()
        self.level = self.level + 1
        if self.level%self.evolve == 0:
            return self.level, "Evolved!"
        else:
            return self.level

    def attack_up(self):
        self.attack = self.attack + self.attack_boost
        return self.attack

    def defense_up(self):
        self.defense = self.defense + self.defense_boost
        return self.defense

    def health_up(self):
        self.health = self.health + self.health_boost
        return self.health

    def update(self):
        self.health_boost = 5
        self.attack_boost = 3
        self.defense_boost = 2
        self.evolve = 10

    def __str__(self):
        return "Pokemon name: {}, Type: {}, Level: {}".format(self.name, self.p_type, self.level)

class Grass_Pokemon(Pokemon):
    attack = 15
    defense = 14
    health = 12
    p_type = "Grass"

    def update(self):
        self.health_boost = 6
        self.attack_boost = 2
        self.defense_boost = 3
        self.evolve = 12

    def moves(self):
        self.p_moves = ["razor leaf", "synthesis", "petal dance"]   
```

</details>

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>

3. Along with the <font color=red>Pokemon</font> parent class, we have also provided several subclasses. Write another method in the parent class that will be inherited by the subclasses. Call it <font color=red>opponent</font>. It should return which type of pokemon the current type is weak and strong against, as a tuple.

  * __Grass__ is weak against Fire and strong against Water
  * __Ghost__ is weak against Dark and strong against Psychic
  * __Fire__ is weak against Water and strong against Grass
  * __Flying__ is weak against Electric and strong against Fighting

  For example, if the <font color=red>p_type</font> of the subclass is <font color=red>'Grass'</font>, <font color=red>.opponent()</font> should return the tuple <font color=red>('Fire', 'Water')</font>

In [5]:
class Pokemon():
    attack = 12
    defense = 10
    health = 15
    p_type = "Normal"

    def __init__(self, name,level = 5):
        self.name = name
        self.level = level
        self.weak = "Normal"
        self.strong = "Normal"

    def train(self):
        self.update()
        self.attack_up()
        self.defense_up()
        self.health_up()
        self.level = self.level + 1
        if self.level%self.evolve == 0:
            return self.level, "Evolved!"
        else:
            return self.level

    def attack_up(self):
        self.attack = self.attack + self.attack_boost
        return self.attack

    def defense_up(self):
        self.defense = self.defense + self.defense_boost
        return self.defense

    def health_up(self):
        self.health = self.health + self.health_boost
        return self.health

    def update(self):
        self.health_boost = 5
        self.attack_boost = 3
        self.defense_boost = 2
        self.evolve = 10

    def __str__(self):
        self.update()
        return "Pokemon name: {}, Type: {}, Level: {}".format(self.name, self.p_type, self.level)

class Grass_Pokemon(Pokemon):
    attack = 15
    defense = 14
    health = 12
    p_type = "Grass"

    def update(self):
        self.health_boost = 6
        self.attack_boost = 2
        self.defense_boost = 3
        self.evolve = 12

class Ghost_Pokemon(Pokemon):
    p_type = "Ghost"

    def update(self):
        self.health_boost = 3
        self.attack_boost = 4
        self.defense_boost = 3

class Fire_Pokemon(Pokemon):
    p_type = "Fire"

class Flying_Pokemon(Pokemon):
    p_type = "Flying"

<details><summary>Click here for a solution</summary>
<div class="alert alert-block alert-success" style="margin-top: 20px">
<font color=black>
    
```python
class Pokemon():
    attack = 12
    defense = 10
    health = 15
    p_type = "Normal"

    def __init__(self, name,level = 5):
        self.name = name
        self.level = level
        self.weak = "Normal"
        self.strong = "Normal"

    def train(self):
        self.update()
        self.attack_up()
        self.defense_up()
        self.health_up()
        self.level = self.level + 1
        if self.level%self.evolve == 0:
            return self.level, "Evolved!"
        else:
            return self.level

    def attack_up(self):
        self.attack = self.attack + self.attack_boost
        return self.attack

    def defense_up(self):
        self.defense = self.defense + self.defense_boost
        return self.defense

    def health_up(self):
        self.health = self.health + self.health_boost
        return self.health

    def update(self):
        self.health_boost = 5
        self.attack_boost = 3
        self.defense_boost = 2
        self.evolve = 10

    def __str__(self):
        self.update()
        return "Pokemon name: {}, Type: {}, Level: {}".format(self.name, self.p_type, self.level)

class Grass_Pokemon(Pokemon):
    attack = 15
    defense = 14
    health = 12
    p_type = "Grass"

    def update(self):
        self.health_boost = 6
        self.attack_boost = 2
        self.defense_boost = 3
        self.evolve = 12

class Ghost_Pokemon(Pokemon):
    p_type = "Ghost"

    def update(self):
        self.health_boost = 3
        self.attack_boost = 4
        self.defense_boost = 3

class Fire_Pokemon(Pokemon):
    p_type = "Fire"

class Flying_Pokemon(Pokemon):
    p_type = "Flying"   
```

</details>

<a id='22.8'></a>
## 22.8. Project - Wheel of Python
[Back to top](#Top)

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>

This project will take you through the process of implementing a simplified version of the game _Wheel of Fortune_. Here are the rules of our game:

  * __There are <font color=red>num_human</font> human players and <font color=red>num_computer</font> computer players.__
      * Every player has some amount of money ($0 at the start of the game)
      * Every player has a set of prizes (none at the start of the game)  
      
      
  * __The goal is to guess a phrase within a category. For example:__
      * Category: __Artist & Song__
      * Phrase: __Whitney Houston’s I Will Always Love You__
      
      
  * __Players see the category and an obscured version of the phrase where every alphanumeric  
  character in the phrase starts out as hidden (using underscores: <font color=red>_</font> ):__
    * Category: Artist & Song
    * Phrase: <font color=red>________ ________'_ _ _______ _________ _____ ____</font>


  * Note that case (capitalization) does not matter
  
  
  * __During their turn, every player spins the wheel to determine a prize amount and:__
    * __If the wheel lands on a cash square, players may do one of three actions:__  
      * __Guess any letter that hasn’t been guessed by typing a letter (a-z)__  
        * Vowels (a, e, i, o, u) cost \$250 to guess and can’t be guessed if the player doesn’t have enough money. All other letters are “free” to guess
        * The player can guess any letter that hasn’t been guessed and gets that cash amount for every time that letter appears in the phrase
        * If there is a prize, the user also gets that prize (in addition to any prizes they already had)
        * If the letter does appear in the phrase, the user keeps their turn. Otherwise, it’s the next player’s turn
        * __Example: The user lands on \$500 and guesses ‘W’__  
          * There are three W’s in the phrase, so the player wins $1500
      * __Guess the complete phrase by typing a phrase (anything over one character that isn’t ‘pass’)__
        * If they are correct, they win the game
        * If they are incorrect, it is the next player’s turn
      * __Pass__ their turn by entering <font color=red>'pass'</font>
    * If the wheel lands on **“lose a turn”**, the player loses their turn and the game moves on to the next player
    * If the wheel lands on **“bankrupt”**, the player loses their turn and loses their money but they keep all of the prizes they have won so far.


  * The game continues until the entire phrase is revealed (or one player guesses the complete phrase)
  
––––

First, let’s learn about a few functions and methods that we’ll use along the way to do this project. There are no questions to answer in the next four active code windows. They are just here to introduce you to some functions and methods that you may not be aware of. The active code window that starts with “Part A” is where you are first asked to complete code.
  
––––

1. The <font color=red>time.sleep(s)</font> function (from the <font color=red>time</font> module) delays execution of the next line of code for **s** seconds. You’ll find that we can build a little suspense during gameplay with some well-placed delays. The game can also be easier for users to understand if not everything happens instantly.

In [7]:
import time

for x in range(2, 6):
    print('Sleep {} seconds..'.format(x))
    time.sleep(x) # "Sleep" for x seconds
print('Done!')

Sleep 2 seconds..
Sleep 3 seconds..
Sleep 4 seconds..
Sleep 5 seconds..
Done!


<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>
    
2. The <font color=red>random</font> module includes several useful methods for generating and using random numbers, including:

  * <font color=red>random.randint(min, max)</font> generates a random number between <font color=red>min</font> and <font color=red>max</font> (inclusive)
  * <font color=red>random.choice(L)</font> selects a random item from the list <font color=red>L</font>

In [ ]:
import random

rand_number = random.randint(1, 10)
print('Random number between 1 and 10: {}'.format(rand_number))

letters = [letter for letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ']
rand_letter = random.choice(letters)
print('Random letter: {}'.format(rand_letter))

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>
 
3. There are also several string methods that we haven’t gone over in detail but will use for this project:

  * <font color=red>.upper()</font> converts a string to uppercase (the opposite is <font color=red>.lower()</font>)
  * <font color=red>.count(s)</font> counts how many times the string <font color=red>s</font> occurs inside of a larger string

In [ ]:
myString = 'Hello, World! 123'

print(myString.upper()) # HELLO, WORLD! 123
print(myString.lower()) # hello, world! 123
print(myString.count('l')) # 3

s = 'python is pythonic'
print(s.count('python')) # 2

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>
 
4. We’re going to define a few useful methods for you:

  * <font color=red>getNumberBetween(prompt, min, max))</font> repeatedly asks the user for a number between <font color=red>min</font> and   <font color=red>max</font> with the prompt <font color=red>prompt</font>
  * <font color=red>spinWheel()</font> simulates spinning the wheel and returns a dictionary with a random prize
  * <font color=red>getRandomCategoryAndPhrase()</font> returns a tuple with a random category and phrase for players to guess
  * <font color=red>obscurePhrase(phrase, guessed)</font> returns a tuple with a random category and phrase for players to guess

  Take some time to read their implementations below.

In [ ]:
import json
import random
import time

LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

# Repeatedly asks the user for a number between min & max (inclusive)
def getNumberBetween(prompt, min, max):
    userinp = input(prompt) # ask the first time

    while True:
        try:
            n = int(userinp) # try casting to an integer
            if n < min:
                errmessage = 'Must be at least {}'.format(min)
            elif n > max:
                errmessage = 'Must be at most {}'.format(max)
            else:
                return n
        except ValueError: # The user didn't enter a number
            errmessage = '{} is not a number.'.format(userinp)

        # If we haven't gotten a number yet, add the error message
        # and ask again
        userinp = input('{}\n{}'.format(errmessage, prompt))

# Spins the wheel of fortune wheel to give a random prize
# Examples:
#    { "type": "cash", "text": "$950", "value": 950, "prize": "A trip to Ann Arbor!" },
#    { "type": "bankrupt", "text": "Bankrupt", "prize": false },
#    { "type": "loseturn", "text": "Lose a turn", "prize": false }
def spinWheel():
    with open("wheel.json", 'r') as f:
        wheel = json.loads(f.read())
        return random.choice(wheel)

# Returns a category & phrase (as a tuple) to guess
# Example:
#     ("Artist & Song", "Whitney Houston's I Will Always Love You")
def getRandomCategoryAndPhrase():
    with open("phrases.json", 'r') as f:
        phrases = json.loads(f.read())

        category = random.choice(list(phrases.keys()))
        phrase   = random.choice(phrases[category])
        return (category, phrase.upper())

# Given a phrase and a list of guessed letters, returns an obscured version
# Example:
#     guessed: ['L', 'B', 'E', 'R', 'N', 'P', 'K', 'X', 'Z']
#     phrase:  "GLACIER NATIONAL PARK"
#     returns> "_L___ER N____N_L P_RK"

def obscurePhrase(phrase, guessed):
    rv = ''
    for s in phrase:
        if (s in LETTERS) and (s not in guessed):
            rv = rv+'_'
        else:
            rv = rv+s
    return rv

# Returns a string representing the current state of the game
def showBoard(category, obscuredPhrase, guessed):
    return """
Category: {}
Phrase:   {}
Guessed:  {}""".format(category, obscuredPhrase, ', '.join(sorted(guessed)))

category, phrase = getRandomCategoryAndPhrase()

guessed = []
for x in range(random.randint(10, 20)):
    randomLetter = random.choice(LETTERS)
    if randomLetter not in guessed:
        guessed.append(randomLetter)

print("getRandomCategoryAndPhrase()\n -> ('{}', '{}')".format(category, phrase))

print("\n{}\n".format("-"*5))

print("obscurePhrase('{}', [{}])\n -> {}".format(phrase, ', '.join(["'{}'".format(c) for c in guessed]), obscurePhrase(phrase, guessed)))

print("\n{}\n".format("-"*5))

obscured_phrase = obscurePhrase(phrase, guessed)
print("showBoard('{}', '{}', [{}])\n -> {}".format(phrase, obscured_phrase, ','.join(["'{}'".format(c) for c in guessed]), showBoard(phrase, obscured_phrase, guessed)))

print("\n{}\n".format("-"*5))

num_times_to_spin = random.randint(2, 5)
print('Spinning the wheel {} times (normally this would just be done once per turn)'.format(num_times_to_spin))

for x in range(num_times_to_spin):
    print("\n{}\n".format("-"*2))
    print("spinWheel()")
    print(spinWheel())


print("\n{}\n".format("-"*5))

print("In 2 seconds, will run getNumberBetween('Testing getNumberBetween(). Enter a number between 1 and 10', 1, 10)")

time.sleep(2)

print(getNumberBetween('Testing getNumberBetween(). Enter a number between 1 and 10', 1, 10))

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>
    
### Part A: WOFPlayer

We’re going to start by defining a class to represent a Wheel of Fortune player, called <font color=red>WOFPlayer</font>. Every instance of <font color=red>WOFPlayer</font> has three instance variables:

  * <font color=red>.name</font>: The name of the player (should be passed into the constructor)
  * <font color=red>.prizeMoney</font>: The amount of prize money for this player (an integer, initialized to <font color=red>0</font>)
  * <font color=red>.prizes</font>: The prizes this player has won so far (a list, initialized to <font color=red>[]</font>)  
  
Of these instance variables, only <font color=red>name</font> should be passed into the constructor.

It should also have the following methods (note: we will exclude <font color=red>self</font> in our descriptions):

  * <font color=red>.addMoney(amt)</font>: Add <font color=red>amt</font> to <font color=red>self.prizeMoney</font>
  * <font color=red>.goBankrupt()</font>: Set <font color=red>self.prizeMoney</font> to <font color=red>0</font>
  * <font color=red>.addPrize(prize)</font>: Append <font color=red>prize</font> to <font color=red>self.prizes</font>
  * <font color=red>.\_\_str\_\_()</font>: __Returns the player’s name and prize money in the following format:__
    * <font color=red>Steve (\$1800)</font> (for a player with instance variables <font color=red>.name == 'Steve'</font> and <font color=red>prizeMoney == 1800</font>)


### Part B: WOFHumanPlayer
Next, we’re going to define a class named <font color=red>WOFHumanPlayer</font>, which should inherit from <font color=red>WOFPlayer</font> (part A). This class is going to represent a human player. In addition to having all of the instance variables and methods that <font color=red>WOFPlayer</font> has, <font color=red>WOFHumanPlayer</font> should have an additional method:

  * <font color=red>.getMove(category, obscuredPhrase, guessed)</font>: Should ask the user to enter a move (using <font color=red>input()</font>) and __return whatever string they entered.__
  
  
<font color=red>.getMove()</font>’s prompt should be:

In [ ]:
{name} has ${prizeMoney}

Category: {category}
Phrase:  {obscured_phrase}
Guessed: {guessed}

Guess a letter, phrase, or type 'exit' or 'pass':

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>
    
For example:

In [ ]:
Steve has $200

Category: Places
Phrase: _L___ER N____N_L P_RK
Guessed: B, E, K, L, N, P, R, X, Z

Guess a letter, phrase, or type 'exit' or 'pass':

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>
    
The user can then enter:

  * <font color=red>'exit'</font> to exit the game
  * <font color=red>'pass'</font> to skip their turn
  * a single character to guess that letter
  * a complete phrase (a multi-character phrase other than <font color=red>'exit'</font> or <font color=red>'pass'</font>) to guess that phrase

Note that <font color=red>.getMove()</font> __does not__ need to enforce anything about the user’s input; that will be done via the game logic that we define in the next ActiveCode window.

### Part C: WOFComputerPlayer

Finally, we’re going to define a class named <font color=red>WOFComputerPlayer</font>, which should inherit from <font color=red>WOFPlayer</font> (part A). This class is going to represent a computer player.

Every computer player will have a <font color=red>difficulty</font> instance variable. Players with a higher <font color=red>difficulty</font> generally play “better”. There are many ways to implement this. We’ll do the following:

  * If there aren’t any possible letters to choose (for example: if the last character is a vowel but this player doesn’t have enough to guess a vowel), we’ll <font color=red>'pass'</font>
  * __Otherwise, semi-randomly decide whether to make a “good” move or a “bad” move on a given turn (a higher difficulty should make it more likely for the player to make a “good” move)__
    * To make a “bad” move, we’ll randomly decide on a possible letter.
    * To make a “good” move, we’ll choose a letter according to their overall frequency in the English language.
    
    
In addition to having all of the instance variables and methods that <font color=red>WOFPlayer</font> has, <font color=red>WOFComputerPlayer</font> should have:

**Class variable**
  * <font color=red>.SORTED_FREQUENCIES</font>: Should be set to <font color=red>'ZQXJKVBPYGFWMUCLDRHSNIOATE'</font>, which is a list of English   characters sorted from least frequent (<font color=red>'Z'</font>) to most frequent (<font color=red>'E'</font>). We’ll use this when trying to make a “good” move.

**Additional Instance variable**
  * <font color=red>.difficulty</font>: The level of difficulty for this computer (should be passed as the second argument into the constructor after <font color=red>.name</font>)

**Methods**

  * <font color=red>.smartCoinFlip()</font>: This method will help us decide semi-randomly whether to make a “good” or “bad” move. A higher difficulty should make us more likely to make a “good” move. Implement this by choosing a random number between <font color=red>1</font> and <font color=red>10</font> using <font color=red>random.randint(1, 10)</font> (see above) and returning <font color=red>True</font> if that random number is greater than self.difficulty. If the random number is less than or equal to <font color=red>self.difficulty</font>, return <font color=red>False</font>.
  * <font color=red>.getPossibleLetters(guessed)</font>: This method should return a list of letters that can be guessed.
    * These should be characters that are in <font color=red>LETTERS</font> (<font color=red>'ABCDEFGHIJKLMNOPQRSTUVWXYZ'</font>) but __not__ in the <font color=red>guessed</font> parameter.
    * Additionally, if this player doesn’t have enough prize money to guess a vowel (variable  
    <font color=red>VOWEL_COST</font> set to <font color=red>250</font>), then vowels (variable <font color=red>VOWELS</font> set to <font color=red>'AEIOU'</font>) should not be included
  * <font color=red>.getMove(category, obscuredPhrase, guessed)</font>: __Should return a valid move.__
    * Use the <font color=red>.getPossibleLetters(guessed)</font> method described above.
    * If there aren’t any letters that can be guessed (this can happen if the only letters left to guess are vowels and the player doesn’t have enough for vowels), return <font color=red>'pass'</font>
    * __Use the <font color=red>.smartCoinFlip()</font> method to decide whether to make a “good” or a “bad” move__
      * If making a “good” move (<font color=red>.smartCoinFlip()</font> returns <font color=red>True</font>), then return the most frequent (highest index in <font color=red>.SORTED_FREQUENCIES</font>) possible character
      * If making a “bad” move (<font color=red>.smartCoinFlip()</font> returns <font color=red>False</font>), then return a random character from the set of possible characters (use <font color=red>random.choice()</font>)

In [ ]:
VOWEL_COST = 250
LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
VOWELS = 'AEIOU'

# Write the WOFPlayer class definition (part A) here

# Write the WOFHumanPlayer class definition (part B) here

# Write the WOFComputerPlayer class definition (part C) here

<details><summary>Click here for a solution</summary>
<div class="alert alert-block alert-success" style="margin-top: 20px">
<font color=black>
    
```python
VOWEL_COST = 250
LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
VOWELS = 'AEIOU'

# Write the WOFPlayer class definition (part A) here
class WOFPlayer:    
    def __init__(self, name):
        self.name = name
        self.prizeMoney = 0
        self.prizes = []
        
    def addMoney(self, amt):
        self.prizeMoney += amt
              
    def goBankrupt(self):
        self.prizeMoney = 0  
        
    def addPrize(self, prize):
        self.prizes.append(prize)  
        
    def __str__(self):
        return '{} (${})'.format(self.name, self.prizeMoney)
    
# Write the WOFHumanPlayer class definition (part B) here
class WOFHumanPlayer(WOFPlayer):   
    
    def getMove(self, category, obscuredPhrase, guessed):
        print('{} has ${}'.format(self.name, self.prizeMoney))
        print("{}".format(showBoard(category, obscuredPhrase, guessed)))
        answer = input("Guess a letter, phrase, or type 'exit' or 'pass': ")
        return answer
    
# Write the WOFComputerPlayer class definition (part C) here
class WOFComputerPlayer(WOFPlayer):   
    
    SORTED_FREQUENCIES = 'ZQXJKVBPYGFWMUCLDRHSNIOATE'
    
    def __init__(self, name, difficulty):
        WOFPlayer.__init__(self, name)
        self.difficulty = difficulty
        
    def smartCoinFlip(self):
        if random.randint(1, 10) > self.difficulty:
            return True
        else:
            return False
        
    def getPossibleLetters(self, guessed):     
        possibleLetters = []
        for letter in LETTERS:
            if letter not in guessed:
                if self.prizeMoney < VOWEL_COST and letter in VOWELS:
                    continue
                else:
                    possibleLetters.append(letter)
        return possibleLetters
        
    def getMove(self, category, obscuredPhrase, guessed):
        validMoves = self.getPossibleLetters(guessed)
        if validMoves == []:
            return 'pass'
        if self.smartCoinFlip():
            for n in range(-1, -len(validMoves), -1):
                if self.SORTED_FREQUENCIES[n] not in validMoves:
                    continue
                else: 
                    return self.SORTED_FREQUENCIES[n]
        else:
            return random.choice(validMoves)  
```

</details>

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>
    

**Putting it together: Wheel of Python**

Below is the game logic for the rest of the “Wheel of Python” game. We have implemented most of the game logic. __Start by carefully reading this code and double checking that it all makes sense__.

Then, paste your code from the previous code window in the correct places below.

**Note 1:** we added the following code to ensure that the Python interpreter gives our game time to run:

In [ ]:
import sys
sys.setExecutionLimit(600000)

<div class="alert alert-block alert-warning" style="margin-top: 20px">
<font color=black>
    
<font color=red>sys.setExecutionLimit(ms)</font> says that we should be able to run our program for <font color=red>ms</font> milliseconds before it gets stopped automatically.

**Note 2:** As you play, you will need to keep scrolling down to follow the game.

In [ ]:
# PASTE YOUR WOFPlayer CLASS (from part A) HERE
# PASTE YOUR WOFHumanPlayer CLASS (from part B) HERE
# PASTE YOUR WOFComputerPlayer CLASS (from part C) HERE


import sys
sys.setExecutionLimit(600000) # let this take up to 10 minutes

import json
import random
import time

LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
VOWELS  = 'AEIOU'
VOWEL_COST  = 250

# Repeatedly asks the user for a number between min & max (inclusive)
def getNumberBetween(prompt, min, max):
    userinp = input(prompt) # ask the first time

    while True:
        try:
            n = int(userinp) # try casting to an integer
            if n < min:
                errmessage = 'Must be at least {}'.format(min)
            elif n > max:
                errmessage = 'Must be at most {}'.format(max)
            else:
                return n
        except ValueError: # The user didn't enter a number
            errmessage = '{} is not a number.'.format(userinp)

        # If we haven't gotten a number yet, add the error message
        # and ask again
        userinp = input('{}\n{}'.format(errmessage, prompt))

# Spins the wheel of fortune wheel to give a random prize
# Examples:
#    { "type": "cash", "text": "$950", "value": 950, "prize": "A trip to Ann Arbor!" },
#    { "type": "bankrupt", "text": "Bankrupt", "prize": false },
#    { "type": "loseturn", "text": "Lose a turn", "prize": false }
def spinWheel():
    with open("wheel.json", 'r') as f:
        wheel = json.loads(f.read())
        return random.choice(wheel)

# Returns a category & phrase (as a tuple) to guess
# Example:
#     ("Artist & Song", "Whitney Houston's I Will Always Love You")
def getRandomCategoryAndPhrase():
    with open("phrases.json", 'r') as f:
        phrases = json.loads(f.read())

        category = random.choice(list(phrases.keys()))
        phrase   = random.choice(phrases[category])
        return (category, phrase.upper())

# Given a phrase and a list of guessed letters, returns an obscured version
# Example:
#     guessed: ['L', 'B', 'E', 'R', 'N', 'P', 'K', 'X', 'Z']
#     phrase:  "GLACIER NATIONAL PARK"
#     returns> "_L___ER N____N_L P_RK"
def obscurePhrase(phrase, guessed):
    rv = ''
    for s in phrase:
        if (s in LETTERS) and (s not in guessed):
            rv = rv+'_'
        else:
            rv = rv+s
    return rv

# Returns a string representing the current state of the game
def showBoard(category, obscuredPhrase, guessed):
    return """
Category: {}
Phrase:   {}
Guessed:  {}""".format(category, obscuredPhrase, ', '.join(sorted(guessed)))

# GAME LOGIC CODE
print('='*15)
print('WHEEL OF PYTHON')
print('='*15)
print('')

num_human = getNumberBetween('How many human players?', 0, 10)

# Create the human player instances
human_players = [WOFHumanPlayer(input('Enter the name for human player #{}'.format(i+1))) for i in range(num_human)]

num_computer = getNumberBetween('How many computer players?', 0, 10)

# If there are computer players, ask how difficult they should be
if num_computer >= 1:
    difficulty = getNumberBetween('What difficulty for the computers? (1-10)', 1, 10)

# Create the computer player instances
computer_players = [WOFComputerPlayer('Computer {}'.format(i+1), difficulty) for i in range(num_computer)]

players = human_players + computer_players

# No players, no game :(
if len(players) == 0:
    print('We need players to play!')
    raise Exception('Not enough players')

# category and phrase are strings.
category, phrase = getRandomCategoryAndPhrase()
# guessed is a list of the letters that have been guessed
guessed = []

# playerIndex keeps track of the index (0 to len(players)-1) of the player whose turn it is
playerIndex = 0

# will be set to the player instance when/if someone wins
winner = False

def requestPlayerMove(player, category, guessed):
    while True: # we're going to keep asking the player for a move until they give a valid one
        time.sleep(0.1) # added so that any feedback is printed out before the next prompt

        move = player.getMove(category, obscurePhrase(phrase, guessed), guessed)
        move = move.upper() # convert whatever the player entered to UPPERCASE
        if move == 'EXIT' or move == 'PASS':
            return move
        elif len(move) == 1: # they guessed a character
            if move not in LETTERS: # the user entered an invalid letter (such as @, #, or $)
                print('Guesses should be letters. Try again.')
                continue
            elif move in guessed: # this letter has already been guessed
                print('{} has already been guessed. Try again.'.format(move))
                continue
            elif move in VOWELS and player.prizeMoney < VOWEL_COST: # if it's a vowel, we need to be sure the player has enough
                    print('Need ${} to guess a vowel. Try again.'.format(VOWEL_COST))
                    continue
            else:
                return move
        else: # they guessed the phrase
            return move

while True:
    player = players[playerIndex]
    wheelPrize = spinWheel()

    print('')
    print('-'*15)
    print(showBoard(category, obscurePhrase(phrase, guessed), guessed))
    print('')
    print('{} spins...'.format(player.name))
    time.sleep(2) # pause for dramatic effect!
    print('{}!'.format(wheelPrize['text']))
    time.sleep(1) # pause again for more dramatic effect!

    if wheelPrize['type'] == 'bankrupt':
        player.goBankrupt()
    elif wheelPrize['type'] == 'loseturn':
        pass # do nothing; just move on to the next player
    elif wheelPrize['type'] == 'cash':
        move = requestPlayerMove(player, category, guessed)
        if move == 'EXIT': # leave the game
            print('Until next time!')
            break
        elif move == 'PASS': # will just move on to next player
            print('{} passes'.format(player.name))
        elif len(move) == 1: # they guessed a letter
            guessed.append(move)

            print('{} guesses "{}"'.format(player.name, move))

            if move in VOWELS:
                player.prizeMoney -= VOWEL_COST

            count = phrase.count(move) # returns an integer with how many times this letter appears
            if count > 0:
                if count == 1:
                    print("There is one {}".format(move))
                else:
                    print("There are {} {}'s".format(count, move))

                # Give them the money and the prizes
                player.addMoney(count * wheelPrize['value'])
                if wheelPrize['prize']:
                    player.addPrize(wheelPrize['prize'])

                # all of the letters have been guessed
                if obscurePhrase(phrase, guessed) == phrase:
                    winner = player
                    break

                continue # this player gets to go again

            elif count == 0:
                print("There is no {}".format(move))
        else: # they guessed the whole phrase
            if move == phrase: # they guessed the full phrase correctly
                winner = player

                # Give them the money and the prizes
                player.addMoney(wheelPrize['value'])
                if wheelPrize['prize']:
                    player.addPrize(wheelPrize['prize'])

                break
            else:
                print('{} was not the phrase'.format(move))

    # Move on to the next player (or go back to player[0] if we reached the end)
    playerIndex = (playerIndex + 1) % len(players)

if winner:
    # In your head, you should hear this as being announced by a game show host
    print('{} wins! The phrase was {}'.format(winner.name, phrase))
    print('{} won ${}'.format(winner.name, winner.prizeMoney))
    if len(winner.prizes) > 0:
        print('{} also won:'.format(winner.name))
        for prize in winner.prizes:
            print('    - {}'.format(prize))
else:
    print('Nobody won. The phrase was {}'.format(phrase))

<details><summary>Click here for a solution</summary>
<div class="alert alert-block alert-success" style="margin-top: 20px">
<font color=black>
    
```python
# PASTE YOUR WOFPlayer CLASS (from part A) HERE
class WOFPlayer:    
    def __init__(self, name):
        self.name = name
        self.prizeMoney = 0
        self.prizes = []       
    def addMoney(self, amt):
        self.prizeMoney += amt           
    def goBankrupt(self):
        self.prizeMoney = 0          
    def addPrize(self, prize):
        self.prizes.append(prize)          
    def __str__(self):
        return '{} (${})'.format(self.name, self.prizeMoney)
    
# PASTE YOUR WOFHumanPlayer CLASS (from part B) HERE
class WOFHumanPlayer(WOFPlayer):       
    def getMove(self, category, obscuredPhrase, guessed):
        print('{} has ${}'.format(self.name, self.prizeMoney))
        print("{}".format(showBoard(category, obscuredPhrase, guessed)))
        answer = input("Guess a letter, phrase, or type 'exit' or 'pass': ")
        return answer
    
# PASTE YOUR WOFComputerPlayer CLASS (from part C) HERE
class WOFComputerPlayer(WOFPlayer):   
    
    SORTED_FREQUENCIES = 'ZQXJKVBPYGFWMUCLDRHSNIOATE'
    
    def __init__(self, name, difficulty):
        WOFPlayer.__init__(self, name)
        self.difficulty = difficulty
        
    def smartCoinFlip(self):
        if random.randint(1, 10) > self.difficulty:
            return True
        else:
            return False
        
    def getPossibleLetters(self, guessed):     
        possibleLetters = []
        for letter in LETTERS:
            if letter not in guessed:
                if self.prizeMoney < VOWEL_COST and letter in VOWELS:
                    continue
                else:
                    possibleLetters.append(letter)
        return possibleLetters
        
    def getMove(self, category, obscuredPhrase, guessed):
        validMoves = self.getPossibleLetters(guessed)
        if validMoves == []:
            return 'pass'
        if self.smartCoinFlip():
            for n in range(-1, -len(validMoves), -1):
                if self.SORTED_FREQUENCIES[n] not in validMoves:
                    continue
                else: 
                    return self.SORTED_FREQUENCIES[n]
        else:
            return random.choice(validMoves)

import sys
sys.setExecutionLimit(600000) # let this take up to 10 minutes

import json
import random
import time

LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
VOWELS  = 'AEIOU'
VOWEL_COST  = 250

# Repeatedly asks the user for a number between min & max (inclusive)
def getNumberBetween(prompt, min, max):
    userinp = input(prompt) # ask the first time

    while True:
        try:
            n = int(userinp) # try casting to an integer
            if n < min:
                errmessage = 'Must be at least {}'.format(min)
            elif n > max:
                errmessage = 'Must be at most {}'.format(max)
            else:
                return n
        except ValueError: # The user didn't enter a number
            errmessage = '{} is not a number.'.format(userinp)

        # If we haven't gotten a number yet, add the error message
        # and ask again
        userinp = input('{}\n{}'.format(errmessage, prompt))

# Spins the wheel of fortune wheel to give a random prize
# Examples:
#    { "type": "cash", "text": "$950", "value": 950, "prize": "A trip to Ann Arbor!" },
#    { "type": "bankrupt", "text": "Bankrupt", "prize": false },
#    { "type": "loseturn", "text": "Lose a turn", "prize": false }
def spinWheel():
    with open("wheel.json", 'r') as f:
        wheel = json.loads(f.read())
        return random.choice(wheel)

# Returns a category & phrase (as a tuple) to guess
# Example:
#     ("Artist & Song", "Whitney Houston's I Will Always Love You")
def getRandomCategoryAndPhrase():
    with open("phrases.json", 'r') as f:
        phrases = json.loads(f.read())

        category = random.choice(list(phrases.keys()))
        phrase   = random.choice(phrases[category])
        return (category, phrase.upper())

# Given a phrase and a list of guessed letters, returns an obscured version
# Example:
#     guessed: ['L', 'B', 'E', 'R', 'N', 'P', 'K', 'X', 'Z']
#     phrase:  "GLACIER NATIONAL PARK"
#     returns> "_L___ER N____N_L P_RK"
def obscurePhrase(phrase, guessed):
    rv = ''
    for s in phrase:
        if (s in LETTERS) and (s not in guessed):
            rv = rv+'_'
        else:
            rv = rv+s
    return rv

# Returns a string representing the current state of the game
def showBoard(category, obscuredPhrase, guessed):
    return """
Category: {}
Phrase:   {}
Guessed:  {}""".format(category, obscuredPhrase, ', '.join(sorted(guessed)))

# GAME LOGIC CODE
print('='*15)
print('WHEEL OF PYTHON')
print('='*15)
print('')

num_human = getNumberBetween('How many human players?', 0, 10)

# Create the human player instances
human_players = [WOFHumanPlayer(input('Enter the name for human player #{}'.format(i+1))) for i in range(num_human)]

num_computer = getNumberBetween('How many computer players?', 0, 10)

# If there are computer players, ask how difficult they should be
if num_computer >= 1:
    difficulty = getNumberBetween('What difficulty for the computers? (1-10)', 1, 10)

# Create the computer player instances
computer_players = [WOFComputerPlayer('Computer {}'.format(i+1), difficulty) for i in range(num_computer)]

players = human_players + computer_players

# No players, no game :(
if len(players) == 0:
    print('We need players to play!')
    raise Exception('Not enough players')

# category and phrase are strings.
category, phrase = getRandomCategoryAndPhrase()
# guessed is a list of the letters that have been guessed
guessed = []

# playerIndex keeps track of the index (0 to len(players)-1) of the player whose turn it is
playerIndex = 0

# will be set to the player instance when/if someone wins
winner = False

def requestPlayerMove(player, category, guessed):
    while True: # we're going to keep asking the player for a move until they give a valid one
        time.sleep(0.1) # added so that any feedback is printed out before the next prompt

        move = player.getMove(category, obscurePhrase(phrase, guessed), guessed)
        move = move.upper() # convert whatever the player entered to UPPERCASE
        if move == 'EXIT' or move == 'PASS':
            return move
        elif len(move) == 1: # they guessed a character
            if move not in LETTERS: # the user entered an invalid letter (such as @, #, or $)
                print('Guesses should be letters. Try again.')
                continue
            elif move in guessed: # this letter has already been guessed
                print('{} has already been guessed. Try again.'.format(move))
                continue
            elif move in VOWELS and player.prizeMoney < VOWEL_COST: # if it's a vowel, we need to be sure the player has enough
                    print('Need ${} to guess a vowel. Try again.'.format(VOWEL_COST))
                    continue
            else:
                return move
        else: # they guessed the phrase
            return move


while True:
    player = players[playerIndex]
    wheelPrize = spinWheel()

    print('')
    print('-'*15)
    print(showBoard(category, obscurePhrase(phrase, guessed), guessed))
    print('')
    print('{} spins...'.format(player.name))
    time.sleep(2) # pause for dramatic effect!
    print('{}!'.format(wheelPrize['text']))
    time.sleep(1) # pause again for more dramatic effect!

    if wheelPrize['type'] == 'bankrupt':
        player.goBankrupt()
    elif wheelPrize['type'] == 'loseturn':
        pass # do nothing; just move on to the next player
    elif wheelPrize['type'] == 'cash':
        move = requestPlayerMove(player, category, guessed)
        if move == 'EXIT': # leave the game
            print('Until next time!')
            break
        elif move == 'PASS': # will just move on to next player
            print('{} passes'.format(player.name))
        elif len(move) == 1: # they guessed a letter
            guessed.append(move)

            print('{} guesses "{}"'.format(player.name, move))

            if move in VOWELS:
                player.prizeMoney -= VOWEL_COST

            count = phrase.count(move) # returns an integer with how many times this letter appears
            if count > 0:
                if count == 1:
                    print("There is one {}".format(move))
                else:
                    print("There are {} {}'s".format(count, move))

                # Give them the money and the prizes
                player.addMoney(count * wheelPrize['value'])
                if wheelPrize['prize']:
                    player.addPrize(wheelPrize['prize'])

                # all of the letters have been guessed
                if obscurePhrase(phrase, guessed) == phrase:
                    winner = player
                    break

                continue # this player gets to go again

            elif count == 0:
                print("There is no {}".format(move))
        else: # they guessed the whole phrase
            if move == phrase: # they guessed the full phrase correctly
                winner = player

                # Give them the money and the prizes
                player.addMoney(wheelPrize['value'])
                if wheelPrize['prize']:
                    player.addPrize(wheelPrize['prize'])

                break
            else:
                print('{} was not the phrase'.format(move))

    # Move on to the next player (or go back to player[0] if we reached the end)
    playerIndex = (playerIndex + 1) % len(players)

if winner:
    # In your head, you should hear this as being announced by a game show host
    print('{} wins! The phrase was {}'.format(winner.name, phrase))
    print('{} won ${}'.format(winner.name, winner.prizeMoney))
    if len(winner.prizes) > 0:
        print('{} also won:'.format(winner.name))
        for prize in winner.prizes:
            print('    - {}'.format(prize))
else:
    print('Nobody won. The phrase was {}'.format(phrase))   
```

</details>

Data file: <font color=red>_wheel.json_</font>

In [ ]:
[
    {
        "type": "cash",
        "text": "$950",
        "value": 950,
        "prize": "A trip to Ann Arbor!"
    },
    {
        "type": "bankrupt",
        "text": "Bankrupt",
        "prize": false
    },
    {
        "type": "loseturn",
        "text": "Lose a turn",
        "prize": false
    },
    {
        "type": "cash",
        "text": "$2500",
        "value": 2500,
        "prize": false
    },
    {
        "type": "cash",
        "text": "$900",
        "value": 900,
        "prize": false
    },
    {
        "type": "cash",
        "text": "$700",
        "value": 700,
        "prize": false
    },
    {
        "type": "cash",
        "text": "$600",
        "value": 600,
        "prize": false
    },
    {
        "type": "cash",
        "text": "$800",
        "value": 800,
        "prize": false
    },
    {
        "type": "cash",
        "text": "One Million",
        "value": 1000000,
        "prize": false
    },
    {
        "type": "cash",
        "text": "$650",
        "value": 650,
        "prize": "A brand new car!"
    },
    {
        "type": "cash",
        "text": "900",
        "value": 900,
        "prize": false
    },
    {
        "type": "cash",
        "text": "$700",
        "value": 700,
        "prize": false
    },
    {
        "type": "cash",
        "text": "$600",
        "value": 600,
        "prize": false
    }
]

Data file: <font color=red>_phrases.json_</font>

In [ ]:
{
"Star & Role": [
    "Adam Sandler As Happy Gilmore",
    "Anthony Hopkins As Nixon",
    "Bob Denver As Gilligan",
    "Candice Bergen As Murphy Brown",
    "Don Johnson As Nash Bridges",
    "Eddie Murphy As The Nutty Professor",
    "Elizabeth Taylor & Richard Burton In Cleopatra",
    "Fran Drescher As The Nanny",
    "Jim Carrey As Ace Ventura",
    "Lea Thompson As Caroline In The City",
    "Marlo Thomas As That Girl",
    "Michael Douglas As The American President",
    "Paul Newman As Butch Cassidy",
    "Peter Falk As Columbo",
    "Peter O'Toole As Lawrence Of Arabia",
    "Pierce Brosnan As James Bond",
    "Sally Field As Norma Rae",
    "Sally Field As The Flying Nun",
    "Steve Martin In Father Of The Bride",
    "Telly Savalas As Kojak",
    "Tom Hanks As Forrest Gump",
    "Tom Selleck As Magnum P.I.",
    "Val Kilmer As Batman"
],
"Title": [
    "A Christmas Carol",
    "A Few Good Men",
    "A Passage To India",
    "A Place In The Sun",
    "A Room With A View",
    "A Soldier's Story",
    "A Star Is Born",
    "A Walk In The Clouds",
    "Ace Ventura When Nature Calls",
    "Adam's Rib",
    "Addams Family Values",
    "Airport",
    "Alice's Adventures In Wonderland",
    "Alien Nation",
    "All That Jazz",
    "All's Well That Ends Well",
    "Amadeus",
    "American Gothic",
    "An American Tail",
    "An Affair To Remember",
    "Angels In The Outfield",
    "Animal Farm",
    "Another World",
    "Are You Lonesome Tonight",
    "Baby It's You",
    "Back In The Saddle Again",
    "Barnaby Jones",
    "Barney Miller",
    "Barney & Friends",
    "Batman Forever",
    "Baywatch",
    "Blackboard Jungle",
    "Blame It On Rio",
    "Bless The Beasts And The Children",
    "Blue Hawaii",
    "Body Heat",
    "Body Of Evidence",
    "Braveheart",
    "Breakfast At Tiffany's",
    "Breaking Up Is Hard To Do",
    "Breathless",
    "Bright Lights Big City",
    "Cabaret",
    "Caesar And Cleopatra",
    "Calendar Girl",
    "California Girls",
    "Call Of The Wild",
    "Camelot",
    "Can You Feel The Love Tonight",
    "Caroline In The City",
    "Charles In Charge",
    "Charlie's Angels",
    "Charlotte's Web",
    "Chicago Hope",
    "Chinatown",
    "Citizen Kane",
    "City Slickers",
    "Clear And Present Danger",
    "Cliffhanger",
    "Coal Miner's Daughter",
    "Columbo",
    "Coming Home",
    "Cool Runnings",
    "Corinna Corinna",
    "Coward Of The County",
    "Cybill",
    "Dangerous Liaisons",
    "Dangerous Minds",
    "Dave's World",
    "Death Be Not Proud",
    "Deliverance",
    "Demolition Man",
    "Dick Tracy",
    "Die Hard With A Vengeance",
    "Dinosaurs",
    "Dirty Harry",
    "Doctor Dolittle",
    "Dog Day Afternoon",
    "Don't Drink The Water",
    "Downhill Racer",
    "Dr. Kildare",
    "Dr. Quinn Medicine Woman",
    "Dream On",
    "Driving Miss Daisy",
    "East Of Eden",
    "East Side West Side",
    "Eight Is Enough",
    "Eleanor And Franklin",
    "Escape From New York",
    "Evening At Pops",
    "Falcon Crest",
    "Falling In Love Again",
    "Family Affair",
    "Family Feud",
    "Fantastic Voyage",
    "Father Of The Bride",
    "Five Easy Pieces",
    "Flight Of The Bumblebee",
    "Flipper",
    "Fool For Love",
    "Forbidden Planet",
    "Forrest Gump",
    "Four Weddings And A Funeral",
    "Francis The Talking Mule",
    "Frasier",
    "Free Willy",
    "Full House",
    "Fun With Dick And Jane",
    "Funny Lady",
    "Funny Face",
    "General Hospital",
    "Gentle Ben",
    "Gentleman's Agreement",
    "Get Shorty",
    "Gilligan's Island",
    "Glory Days",
    "Go Ask Alice",
    "Good Morning Vietnam",
    "Goodbye Columbus",
    "Goodbye Norma Jean",
    "Gorillas In The Mist",
    "Grand Hotel",
    "Grease",
    "Groundhog Day",
    "Grumpy Old Men",
    "Hail To The Chief",
    "Hamlet",
    "Hannah And Her Sisters",
    "Happy Trails",
    "Hard Times",
    "Heaven Can Wait",
    "Help Me Make It Through The Night",
    "Here Comes The Bride",
    "Hester Street",
    "High Society",
    "High Noon",
    "Hogan's Heroes",
    "Hollywood Squares",
    "Homicide Life On The Street",
    "Honey I Blew Up The Kid",
    "How The West Was Won",
    "Howard's End",
    "I Am A Fugitive From A Chain Gang",
    "I Dream Of Jeannie",
    "I Love Trouble",
    "I Will Follow Him",
    "I'll Fly Away",
    "I've Got A Lovely Bunch Of Coconuts",
    "If It's Tuesday This Must Be Belgium",
    "In The Heat Of The Night",
    "In The Mood",
    "In The Name Of The Father",
    "Independence Day",
    "Ironside",
    "It Had To Be You",
    "It Takes A Thief",
    "Jack And The Beanstalk",
    "Jonny Quest",
    "Junior",
    "Jurassic Park",
    "Just A Gigolo",
    "Kate And Allie",
    "King Of The Road",
    "Kiss Me Kate",
    "Kiss Of The Spider Woman",
    "Knots Landing",
    "L.A. Law",
    "Lady Sings The Blues",
    "Larry King Live",
    "Lassie Come Home",
    "Last Action Hero",
    "Late Night With Greg Kinnear",
    "Late Show With David Letterman",
    "Law & Order",
    "Leader Of The Pack",
    "Lean On Me",
    "Legal Eagles",
    "Less Than Zero",
    "Let It Be",
    "Let's Dance",
    "Lifestyles Of The Rich And Famous",
    "Like Water For Chocolate",
    "Lilies Of The Field",
    "Little Caesar",
    "Little Giants",
    "Little Man Tate",
    "Little Women",
    "Lonesome Dove",
    "Love Affair",
    "Love American Style",
    "Love And War",
    "Love Of My Life",
    "Main Street",
    "Make Room For Daddy",
    "Manhattan Murder Mystery",
    "Marathon Man",
    "Mary Had A Little Lamb",
    "Masterpiece Theater",
    "Max Headroom",
    "Mayberry R.F.D.",
    "Mchale's Navy",
    "Medical Center",
    "Meet Me In St. Louis",
    "Miami Vice",
    "Mighty Morphin Power Rangers",
    "Milk Money",
    "Money For Nothing",
    "Moon Over Miami",
    "Mortal Kombat",
    "Mr. Roberts",
    "Mr. Saturday Night",
    "Mrs. Doubtfire",
    "Murder One",
    "My Cousin Vinny",
    "My Favorite Martian",
    "My Friend Flicka",
    "My Gal Sal",
    "My Fair Lady",
    "My Heroes Have Always Been Cowboys",
    "My Mother The Car",
    "My Three Sons",
    "Nashville",
    "National Lampoon",
    "Network",
    "Next Stop Greenwich Village",
    "Night Of The Living Dead",
    "Nobody's Fool",
    "Norma Rae",
    "North Dallas Forty",
    "Nothing In Common",
    "Ode To Billie Joe",
    "On Golden Pond",
    "On A Clear Day You Can See Forever",
    "One Day At A Time",
    "One Life To Live",
    "Only The Lonely",
    "Only You",
    "Ordinary People",
    "Our Miss Brooks",
    "Paint Your Wagon",
    "Paper Lion",
    "Patton",
    "Picket Fences",
    "Pillow Talk",
    "Pink Cadillac",
    "Planet Of The Apes",
    "Platoon",
    "Play It Again Sam",
    "Porgy And Bess",
    "Postcards From The Edge",
    "Prelude To A Kiss",
    "Pretty Woman",
    "Private Lives",
    "Prizzi's Honor",
    "Problem Child",
    "Proud Mary",
    "Puff The Magic Dragon",
    "Pulp Fiction",
    "Pulp Fiction",
    "Pygmalion",
    "Quiz Show",
    "Raging Bull",
    "Rear Window",
    "Rebecca",
    "Reservoir Dogs",
    "Ripley's Believe It Or Not",
    "Risky Business",
    "Robin Hood Prince Of Thieves",
    "Rocky",
    "Roman Holiday",
    "Romancing The Stone",
    "Romeo And Juliet",
    "Running On Empty",
    "Saved By The Bell",
    "Scarecrow And Mrs. King",
    "Scent Of A Woman",
    "Sense And Sensibility",
    "Sesame Street",
    "Shall We Dance",
    "She Wore A Yellow Ribbon",
    "Shine On Harvest Moon",
    "Shining Star",
    "Shining Through",
    "Short Cuts",
    "Silver Bells",
    "Sister Act",
    "Six Degrees Of Separation",
    "Sleepless In Seattle",
    "Snow White And The Seven Dwarfs",
    "Some Kind Of Hero",
    "Sophie's Choice",
    "Speed",
    "Stagecoach",
    "Stand By Your Man",
    "Star Search",
    "Star Trek Deep Space Nine",
    "Star Trek Generations",
    "Star Trek Voyager",
    "Stardust Memories",
    "Stargate",
    "Starsky And Hutch",
    "Staying Alive",
    "Still Crazy After All These Years",
    "Stormy Weather",
    "Sunday In The Park",
    "Tales From The Crypt",
    "Taxi Driver",
    "Tender Is The Night",
    "Tender Mercies",
    "Tequila Sunrise",
    "That Girl",
    "The American President",
    "The Apartment",
    "The Bodyguard",
    "The Buddy Holly Story",
    "The Canterbury Tales",
    "The Citadel",
    "The Crying Game",
    "The Diary Of Anne Frank",
    "The Electric Horseman",
    "The Fabulous Baker Boys",
    "The Farmer's Daughter",
    "The Flintstones",
    "The Frugal Gourmet",
    "The Godfather",
    "The Goodbye Girl",
    "The Great Escape",
    "The Greatest Story Ever Told",
    "The Hustler",
    "The Iceman Cometh",
    "The Incredible Hulk",
    "The Incredible Journey",
    "The Jungle Book",
    "The Killing Fields",
    "The Little Rascals",
    "The Luci-Desi Comedy Hour",
    "The Mission",
    "The Money Pit",
    "The Right Stuff",
    "The River Wild",
    "The Santa Clause",
    "The Scarlet Letter",
    "The Shaggy Dog",
    "The Shawshank Redemption",
    "The Simpsons",
    "The Single Guy",
    "The Sound Of Music",
    "The Specialist",
    "The Terminator",
    "The Today Show",
    "The Turning Point",
    "The Unbearable Lightness Of Being",
    "The Verdict",
    "The Wackiest Ship In The Army",
    "The Wonder Years",
    "The X-Files",
    "The Age Of Innocence",
    "The Andy Griffith Show",
    "The Call Of The Wild",
    "The Champ",
    "The Circle Of Life",
    "The Color Of Money",
    "The Color Purple",
    "The Computer Wore Tennis Shoes",
    "The Cosby Show",
    "The Day The Earth Stood Still",
    "The Days And Nights Of Molly Dodd",
    "The Deer Hunter",
    "The Diary Of Anne Frank",
    "The Dirty Dozen",
    "The Gambler",
    "The Grapes Of Wrath",
    "The Green Hornet",
    "The Guns Of Navarone",
    "The Jackie Gleason Show",
    "The Last Boy Scout",
    "The Last Detail",
    "The Last Picture Show",
    "The Last Time I Saw Paris",
    "The Lion King",
    "The Little Prince",
    "The Living End",
    "The Lone Ranger",
    "The Long Goodbye",
    "The Lucy Show",
    "The Main Event",
    "The Man In The Iron Mask",
    "The Man Who Knew Too Much",
    "The Man Who Loved Cat Dancing",
    "The Mask",
    "The Member Of The Wedding",
    "The Mod Squad",
    "The Mommies",
    "The Mouse That Roared",
    "The Naked Truth",
    "The Nanny",
    "The Next Karate Kid",
    "The Night Has A Thousand Eyes",
    "The Nutty Professor",
    "The One That Got Away",
    "The Other Side Of The Mountain",
    "The Outer Limits",
    "The Patty Duke Show",
    "The Price Is Right",
    "The Pride Of The Yankees",
    "The Prince Of Tides",
    "The Real World",
    "The Red Pony",
    "The Sand Pebbles",
    "The Seven Percent Solution",
    "The Spy Who Came In From The Cold",
    "The Spy Who Loved Me",
    "The Sun Also Rises",
    "The Swiss Family Robinson",
    "The Sword In The Stone",
    "The Three Musketeers",
    "The Usual Suspects",
    "The Waltons",
    "The War",
    "The War Of The Roses",
    "The Way We Were",
    "The Wind In The Willows",
    "The Wizard Of Oz",
    "The Year Of Living Dangerously",
    "The Yellow Rose Of Texas",
    "The Young And The Restless",
    "Thelma & Louise",
    "There's No Business Like Show Business",
    "Three Days Of The Condor",
    "Till We Meet Again",
    "Timecop",
    "To Have And Have Not",
    "Tootsie",
    "Top Gun",
    "Top Hat",
    "Torn Between Two Lovers",
    "Troop Beverly Hills",
    "True Confessions",
    "True Lies",
    "Twin Peaks",
    "Twist And Shout",
    "Twister",
    "Two Gentlemen Of Verona",
    "Two Years Before The Mast",
    "Unforgiven",
    "Unsolved Mysteries",
    "Up The Down Staircase",
    "Voodoo Lounge",
    "Voyage To The Bottom Of The Sea",
    "Walking Tall",
    "Waterworld",
    "Weekend At Bernie's",
    "Welcome Back Kotter",
    "What's Eating Gilbert Grape",
    "What's My Line",
    "What's Love Got To Do With It",
    "What's New Pussycat",
    "When Dinosaurs Ruled The Earth",
    "When Harry Met Sally",
    "When Will I Be Loved",
    "When You Wish Upon A Star",
    "Where The Boys Are",
    "While You Were Sleeping",
    "White Men Can't Jump",
    "White Nights",
    "Who Framed Roger Rabbit",
    "Whose Life Is It Anyway",
    "Wild Thing",
    "Witness For The Prosecution",
    "Woman Of The Year",
    "Wonder Woman",
    "Working Girl",
    "Yellow Submarine",
    "You Can't Hurry Love"
],
"Headline": [
    "Bill Clinton Elected For Second Term",
    "Charles & Diana Finalize Divorce",
    "Elvis Enlists In The U.S. Army",
    "Hubble Telescope Sends Dramatic Space Photos",
    "Jfk Jr. Secretly Weds Carolyn Bessette",
    "Madonna Gives Birth To A Baby Girl",
    "Michael Jackson & Lisa Marie Presley Divorce",
    "Sonny Bono Elected Mayor Of Palm Springs",
    "Thousands Log On To The Internet",
    "U.S. Sends Chimp Into Outer Space",
    "Watergate Scandal Forces Nixon To Resign",
    "Yankees Defeat Braves To Win The World Series"
],
"Things": [
    "Action-Adventure Films",
    "Alfalfa Sprouts",
    "Asterisks",
    "Autumn Leaves",
    "Avocados Mangoes & Grapefruit",
    "Bacon And Eggs",
    "Bacon Bits",
    "Bagel With Lox And Cream Cheese",
    "Bartlett Pears",
    "Blueberry Muffins",
    "Boxing Gloves",
    "Bread Crumbs",
    "Breath Mints",
    "Bubbles",
    "Buffalo Chicken Wings",
    "Building Blocks",
    "Candied Yams With Marshmallows",
    "Checkers",
    "Chestnuts",
    "Chocolate Chips",
    "Circles",
    "Civil Rights",
    "Coat And Tie",
    "Coattails",
    "Coffee With Cream And Two Sugars",
    "Collectable Coins",
    "Cowboy Boots & Spurs",
    "Crab Cakes",
    "Crayons",
    "Detailed Descriptions",
    "Dewdrops",
    "Dill Rosemary & Thyme",
    "Dining Room Chairs",
    "Directions",
    "Dirt And Grime",
    "Dirty Dishes",
    "Dog Tags",
    "Dominoes",
    "Double Doors",
    "Economic Indicators",
    "Endangered Species",
    "Espresso Cappuccino & Decaf Coffee",
    "Eyelids",
    "Fangs",
    "Farm Animals",
    "Fiber Optics",
    "Filters",
    "Final Exams",
    "Fireworks",
    "Flames",
    "Flesh And Blood",
    "Footlights",
    "Forget-Me-Nots",
    "Freckles",
    "Fringe Benefits",
    "Funny Papers",
    "Gale Force Winds",
    "Golf Clubs",
    "Goods And Services",
    "Groceries",
    "Guard Dogs",
    "Guest Towels",
    "Hammer And Nails",
    "Hand-Me-Down Clothes",
    "Handcuffs",
    "Hash Brown Potatoes",
    "Hearts Diamonds Clubs & Spades",
    "Hidden Compartments",
    "High Marks",
    "House Keys",
    "Household Hints",
    "Houseplants",
    "Hush Puppies",
    "Illustrations",
    "Incentives",
    "Investments",
    "Jumping Jacks",
    "Kidney Beans",
    "Ladyfingers",
    "Leather Gloves",
    "Limited Resources",
    "Living Room Drapes",
    "Lyrics",
    "Macaroni And Cheese",
    "Missed Opportunities",
    "Mistakes",
    "Morning Exercises",
    "Mountain Climbing Equipment",
    "Municipal Bonds",
    "Mushrooms",
    "Musical Instruments",
    "Napkins",
    "Newspaper Articles",
    "Numbers",
    "Numerator And Denominator",
    "Nursery Rhymes",
    "Nylon Stockings",
    "Office Furniture",
    "Orange Blossoms",
    "Oranges",
    "Overhead Lights",
    "Pancakes",
    "Pies And Tarts",
    "Pine Needles",
    "Pine Nuts",
    "Pink Elephants",
    "Pins And Needles",
    "Plants",
    "Polo Ponies",
    "Prayer Beads",
    "Push Pins",
    "Questions",
    "Radial Tires",
    "Record Books",
    "Reference Books",
    "Regular Examinations",
    "Replacement Parts",
    "Restrictions",
    "Rhinestones",
    "Rice Cakes",
    "Riding Boots",
    "Roman Numerals",
    "Saddle Bags",
    "Sale Merchandise",
    "Salt And Pepper",
    "Sand Dunes",
    "Sandals",
    "Sandy Beaches",
    "Saplings",
    "Scallions",
    "Scallops",
    "Seashells",
    "Shampoo & Conditioner",
    "Sideburns",
    "Sled Dogs",
    "Soybeans",
    "Spade Shovel & Hoe",
    "Spare Parts",
    "Stained Glass Windows",
    "Standard Requirements",
    "Statistics",
    "Stereo Components",
    "Streamers",
    "Student Lectures",
    "Sugar Cubes",
    "Syllables",
    "Tax Deductions",
    "Telephone Directories",
    "Television Networks",
    "Toothpicks",
    "Tree Branches",
    "Tropical Flowers",
    "Twin Rollaway & Double Beds",
    "Used Automobiles",
    "Vegetables",
    "Verbal Commands",
    "Vital Statistics",
    "Vitamins And Minerals",
    "Watercolors",
    "Weather Forecasts",
    "Weeks And Months",
    "Wild Beasts",
    "Wild Horses",
    "Wooden Shoes",
    "Written Estimates",
    "Yellow Daisies",
    "Zoo Animals"
],
"Artist & Song": [
    "Barbra Streisand's Memory",
    "Barbra Streisand's The Way We Were",
    "Billy Joel's The Piano Man",
    "Bing Crosby's White Christmas",
    "Bob Hope's Thanks For The Memory",
    "Bruce Springsteen's Born In The USA",
    "Elton John's Goodbye Yellow Brick Road",
    "Frank Sinatra's My Way",
    "James Taylor's Yo've Got A Friend",
    "John Lennon's Imagine",
    "Judy Garland's Over The Rainbow",
    "Liza Minelli's New York New York",
    "Louis Armstrong's Hello Dolly",
    "Paul Simon's Slip Slidin' Away",
    "The Beatles' Hey Jude",
    "The Village People's YMCA",
    "Whitney Houston's I Will Always Love You"
],
"Fictional Character": [
    "Aphrodite",
    "Babar King Of The Elephants",
    "Batman",
    "Betty Rubble",
    "Bugs Bunny",
    "Count Dracula",
    "Daffy Duck",
    "Elmer Fudd",
    "Flash Gordon",
    "Huckleberry Finn",
    "Indiana Jones",
    "Jack Be Nimble",
    "Johnny Appleseed",
    "Lady Macbeth",
    "Leprechaun",
    "Little Red Riding Hood",
    "Mother Goose",
    "Nancy Drew",
    "Olive Oyl",
    "Papa Bear",
    "Perry Mason",
    "Pocahontas",
    "Popeye The Sailor Man",
    "Quick-Draw McGraw",
    "Rip Van Winkle",
    "Simba",
    "Simple Simon",
    "Snow White",
    "The Cheshire Cat",
    "The Frog Prince",
    "The Sandman",
    "The Sheriff Of Nottingham",
    "The Glass Menagerie's Gentleman Caller",
    "The Man In The Moon",
    "Tiny Tim",
    "Tom Sawyer",
    "Unicorn",
    "Walt Kelly's Pogo The Possum",
    "Wee Willie Winkie",
    "Wilma Flintstone",
    "Wise Old Owl",
    "Yosemite Sam"
],
"The Seventies": [
    "Apple Starts Producing Personal Computers",
    "Egypt & Israel Sign Historic Peace Treaty",
    "Evita & A Chorus Line Are Broadway Hits",
    "First Test-Tube Baby Born",
    "Mark Spitz Wins Seven Olympic Gold Medals",
    "The U.S. Celebrates Its Bicentennial",
    "U.S. Signs Treaty Returning Panama Canal"
],
"Show Biz": [
    "Audrey Hepburn Has Breakfast At Tiffany's",
    "Bob Hope Entertains The Troops",
    "Bob Hope & Bing Crosby Star In Road Movies",
    "Comic Relief Farm Aid & Live Aid",
    "Late Night Talk Show Wars",
    "Paramount & Warner Brothers Start TV Networks",
    "Sylvester Stallone Makes Five Rocky Movies",
    "The Beatles Appear On The Ed Sullivan Show",
    "The Muppets Get Their Own TV Show"
],
"The Sixties": [
    "Lyndon Johnson Re-Elected As President",
    "Medicare Provides Aid To The Elderly",
    "National Organization For Women Founded",
    "Soviets Erect Berlin Wall",
    "Thousands Attend Concert At Woodstock"
],
"Classic TV": [
    "Charlie Brown & Snoopy Come To Television",
    "Gilligan & Friends Take A Three-Hour Tour",
    "Gunsmoke Rawhide & Bonanza",
    "How Sweet It Is",
    "I'm So Glad We Had This Time Together",
    "Jack Webb Stars In Dragnet",
    "Lassie & Flipper",
    "Lucy & Ethel Get Into Trouble On I Love Lucy",
    "Mork & Mindy",
    "My Favorite Martian",
    "Oh I Wish I Were An Oscar Mayer Weiner",
    "See The USA In Your Chevrolet",
    "Sid Caesar & Imogene Coca In Your Show Of Shows",
    "Star Trek & Lost In Space",
    "The Jetsons & The Flintstones",
    "The Lone Ranger & Tonto",
    "The Munsters & The Addams Family",
    "Wheel Of Fortune Debuts On Nighttime Television"
],
"Same Name": [
    "Barnaby & Me And Mrs Jones",
    "Barney & Mitch Miller",
    "Bird's & Empty Nest",
    "Breaking & I'll Fly Away",
    "Cat On A Hot Tin & Fiddler On The Roof",
    "Clean-Up & Pancake Batter",
    "Death & Lily Of The Valley",
    "Designing & Little Women",
    "Dog Day & Sunday Afternoon",
    "Eager & Leave It To Beaver",
    "Ella & F. Scott Fitzgerald",
    "Ellis & Fantasy Island",
    "Evening & Window Shade",
    "Fantasy & Gilligan's Island",
    "Flower And Taste Buds",
    "Foul & One-Act Play",
    "Full & Open House",
    "George & Burning Bush",
    "Gilligan's & Treasure Island",
    "Glory & Happy Days",
    "Golf & Cuff Links",
    "Good Morning & North America",
    "Groucho & Karl Marx",
    "Head Over & High Heels",
    "Helen & Sea Hunt",
    "Highway To & Pennies From Heaven",
    "Hugh & Churchill Downs",
    "Human & Mother Nature",
    "Japanese & Volkswagen Beetle",
    "Johnny & Kit Carson",
    "Key & Adam West",
    "Knots & Moon Landing",
    "Liberty & Alexander Graham Bell",
    "Michael & New York",
    "Mister & Kenny Rogers",
    "Monty & Annie Hall",
    "Murphy & James Brown",
    "Natalie & Old King Cole",
    "Neck & Family Ties",
    "Night & Order In The Court",
    "Panama & Root Canal",
    "Piccadilly & Three-Ring Circus",
    "Pool & Great White Shark",
    "President James & Marilyn Monroe",
    "Rodeo & Midnight Cowboy",
    "Root Beer & Orange Bowl Parade Floats",
    "Roy Will & Kenny Rogers",
    "Spike & Robert E. Lee",
    "Sweet & Field Of Dreams",
    "The Deer & Holly Hunter",
    "The Lone & Park Ranger"
],
"Author & Title": [
    "For Whom The Bell Tolls By Ernest Hemingway",
    "Pride & Prejudice By Jane Austen",
    "Tales Of The South Pacific By James Michener",
    "The Great Gatsby By F. Scott Fitzgerald",
    "The Hunt For Red October By Tom Clancy",
    "The Right Stuff By Tom Wolfe",
    "The Tale Of Peter Rabbit By Beatrix Potter",
    "The World According To Garp By John Irving"
],
"Song/Show": [
    "A Whole New World From Aladdin",
    "Colors Of The World From Pocahontas",
    "Don't Cry For Me Argentina From Evita",
    "Getting To Know You From The King And I",
    "I Could Have Danced All Night From My Fair Lady",
    "I Feel Pretty From West Side Story",
    "Memory From Cats",
    "Under The Sea From The Little Mermaid",
    "What I Did For Love From A Chorus Line"
],
"Husband & Wife": [
    "Al & Tipper Gore",
    "Arnold Schwarzenegger & Maria Shriver",
    "Bill & Hillary Rodham Clinton",
    "Bruce Willis & Demi Moore",
    "George & Barbara Bush",
    "Jimmy & Rosalynn Carter",
    "John Travolta & Kelly Preston",
    "John F. Kennedy Jr. & Carolyn Bessette",
    "Paul Newman & Joanne Woodward",
    "Queen Elizabeth & Prince Philip",
    "Ronald & Nancy Reagan",
    "Steven Spielberg & Kate Capshaw",
    "Ted Turner & Jane Fonda",
    "Warren Beatty & Annette Bening"
],
"Nickname": [
    "Billy The Kid",
    "Calamity Jane",
    "Ivan The Terrible",
    "Legs Diamond",
    "Long John Silver",
    "Merry Old England",
    "Miracle Mile",
    "Old Ironsides",
    "Old Man Rhythm",
    "Paddy Wagon",
    "Peter The Great",
    "The Emerald Isle",
    "The Fourth Estate",
    "The City Of Brotherly Love",
    "The Land Of Plenty",
    "The Roaring Twenties",
    "The Wild West",
    "Tin Pan Alley",
    "Whirlybird"
],
"Quotation": [
    "A Bicycle Built For Two",
    "A Legend In His Own Time",
    "A Pocket Full Of Posies",
    "A Poem Lovely As A Tree",
    "A Rolling Stone Gathers No Moss",
    "A Yankee Doodle Do Or Die",
    "All I Have To Do Is Dream",
    "Baby We Were Born To Run",
    "Big Wheels Keep On Turning",
    "Bring Back My Bonny To Me",
    "Bringing In The Sheaves",
    "Clang Clang Clang Went The Trolley",
    "Curses Foiled Again",
    "Deck The Halls With Boughs Of Holly",
    "Deep In The Heart Of Texas",
    "Do The Twist",
    "Doe A Deer A Female Deer",
    "Don't Tread On Me",
    "Four Score And Seven Years Ago",
    "From The Mountains To The Prairies",
    "Gently Down The Stream",
    "Give Us This Day Our Daily Bread",
    "Goodness Gracious Great Balls Of Fire",
    "He's Making A List And Checking It Twice",
    "Healthy Wealthy And Wise",
    "Here We Go Round The Mulberry Bush",
    "Hey Diddle Diddle The Cat And The Fiddle",
    "Hickory Dickory Dock",
    "How Deep Is Your Love",
    "How Do You Spell Relief",
    "How Sweet It Is",
    "I Came I Saw I Conquered",
    "I Cannot Tell A Lie",
    "I Cannot Tell A Lie",
    "I Pledge Allegiance To The Flag",
    "I'm In The Mood For Love",
    "It's A Long Way To Tipperary",
    "Like Sands Through The Hourglass",
    "Little Boy Blue Come Blow Your Horn",
    "Love Thy Neighbor",
    "Love To Love You Baby",
    "Mary Had A Little Lamb",
    "Miles To Go Before I Sleep",
    "My Cup Runneth Over",
    "Never Trust Anyone Over Thirty",
    "No Man Is An Island",
    "No Way To Treat A Lady",
    "O'er The Ramparts We Watched",
    "Oh You Beautiful Doll",
    "Once Upon A Time",
    "Only The Shadow Knows",
    "Pocket Full Of Posies",
    "Rally 'Round The Flag Boys",
    "Reach Out And Touch Someone",
    "Root Root Root For The Home Team",
    "Row Row Row Your Boat",
    "See Spot Run",
    "Seek And Ye Shall Find",
    "Some Guys Have All The Luck",
    "Take My Wife Please",
    "Teach Your Children Well",
    "Thanks For The Memory",
    "The Farmer Takes A Wife",
    "The Fog Comes In On Little Cat Feet",
    "The Meek Shall Inherit The Earth",
    "The Shot Heard Round The World",
    "This Land Was Made For You And Me",
    "Tie A Yellow Ribbon",
    "To Err Is Human To Forgive Divine",
    "To Form A More Perfect Union",
    "Twinkle Twinkle Little Star",
    "We Have Met The Enemy And He Is Us",
    "What's Good For The Goose Is Good For The Gander",
    "When In The Course Of Human Events",
    "Where Seldom Is Heard",
    "Wherefore Art Thou Romeo",
    "Workers Of The World Unite"
],
"The Eighties": [
    "East & West Germany Reunite",
    "John McEnroe Wins Three Wimbledon Titles",
    "MTV Debuts All-Music Format",
    "The Soviet Union Collapses"
],
"Places": [
    "Baggage Claim Areas",
    "Canadian Provinces",
    "Canary Islands",
    "Churches & Synagogues",
    "Islands Of The West Indies",
    "Northern & Southern Suburbs",
    "Samoan Islands",
    "School Zones",
    "Southern States",
    "The Florida Keys",
    "The Windward Islands",
    "The Four Corners Of The Earth",
    "Virgin Islands"
],
"Person - Proper Name": [
    "Abraham Lincoln",
    "Actor Director Clint Eastwood",
    "Actor Ernest Borgnine",
    "Actress Comedian Whoopi Goldberg",
    "Actress Dana Delaney",
    "Actress Heather Locklear",
    "Adlai Stevenson",
    "Al Gore",
    "Albert Einstein",
    "Aldous Huxley",
    "Ambrose Bierce",
    "Anatole France",
    "Andy Rooney",
    "Andy Warhol",
    "Anita Baker",
    "Ann Jillian",
    "Annette Bening",
    "Architect Frank Lloyd Wright",
    "Art Linkletter",
    "Astronaut John Glenn",
    "Astronomer Galileo",
    "Attorney General Janet Reno",
    "Author Herman Melville",
    "Author James Thurber",
    "Aviator Charles Lindbergh",
    "Beatrix Potter",
    "Bertrand Russell",
    "Bill Clinton",
    "Bruce Jenner",
    "Bruce Willis",
    "Calvin Coolidge",
    "Cleopatra",
    "Clint Eastwood",
    "Comedian Ellen Degeneres",
    "Comedian George Carlin",
    "Composer George Gershwin",
    "Country Singer Johnny Cash",
    "Dan Quayle",
    "Daniel Boone",
    "Davy Crockett",
    "Drew Barrymore",
    "Dustin Hoffman",
    "Dwight D. Eisenhower",
    "Dwight Yoakam",
    "Elvis Presley",
    "Eric Clapton",
    "Ernest Borgnine",
    "Fashion Designer Giorgio Armani",
    "Film Director John Huston",
    "Flip Wilson",
    "Florence Nightingale",
    "Former First Lady Barbara Bush",
    "Franklin Delano Roosevelt",
    "George Bernard Shaw",
    "George Bush",
    "George Gershwin",
    "George Washington",
    "Gerald Ford",
    "Giorgio Armani",
    "Godfather Of Soul James Brown",
    "Grandma Moses",
    "Grover Cleveland",
    "Harrison Ford",
    "Harry S Truman",
    "Helen Of Troy",
    "Henry Fonda",
    "Herbert Hoover",
    "Home Improvement Star Tim Allen",
    "Horatio Alger",
    "Indira Gandhi",
    "J. Edgar Hoover",
    "Jack Nicholson",
    "Jackie Gleason",
    "James Dean",
    "Jenny Jones",
    "Jimmy Carter",
    "John Adams",
    "John Barrymore",
    "John Hancock",
    "Jonathan Swift",
    "Julius Caesar",
    "Karl Marx",
    "Leading Lady Mae West",
    "Leading Man Harrison Ford",
    "Lenny Kravitz",
    "Leonardo Da Vinci",
    "Lord Byron",
    "Luciano Pavarotti",
    "Luther Vandross",
    "Lyndon Baines Johnson",
    "Magician David Copperfield",
    "Marilyn Monroe",
    "Martin Luther King",
    "Mary Chapin Carpenter",
    "Melissa Etheridge",
    "Meryl Streep",
    "Mother Teresa",
    "Napoleon Bonaparte",
    "Neil Young",
    "New York Mayor Rudolph Giuliani",
    "News Correspondent Charles Kuralt",
    "Norman Rockwell",
    "Novelist Jack London",
    "Oliver North",
    "Orson Welles",
    "Patty Hearst",
    "Paul Newman",
    "Paul Revere",
    "Peter Gabriel",
    "Phil Donahue",
    "Placido Domingo",
    "Poet Carl Sandburg",
    "President Calvin Coolidge",
    "Prince Rainier Of Monaco",
    "Randy Quaid",
    "Revolutionary Leader John Hancock",
    "Richard Nixon",
    "Robert Stack",
    "Rock Musician Sting",
    "Roseanne",
    "Rosie O'Donnell",
    "Rush Limbaugh",
    "Sam Snead",
    "Scientist Albert Einstein",
    "Sheryl Crow",
    "Singer Bonnie Raitt",
    "Singer Toni Braxton",
    "Socrates",
    "Speaker Of The House Newt Gingrich",
    "Superstar Elton John",
    "Talk Show Host Phil Donahue",
    "Ted Turner",
    "The Dalai Lama",
    "Theodore Roosevelt",
    "Thomas Jefferson",
    "Tom Jones",
    "Tony Bennett",
    "Van Morrison",
    "Warren Beatty",
    "Whoopi Goldberg",
    "Will Rogers",
    "William Shakespeare",
    "Winston Churchill",
    "Woodrow Wilson",
    "Wynonna Judd",
    "Yoko Ono",
    "Zsa Zsa Gabor"
],
"Person": [
    "A Good Soldier",
    "A Handsome Man",
    "A Mature Individual",
    "A Mere Child",
    "A Pain In The Neck",
    "A Professor With Tenure",
    "A Real Crowd-Pleaser",
    "A Real Looker",
    "Acquaintance",
    "Administrator",
    "Angel",
    "Archer",
    "Artist",
    "Aunt",
    "Best Man",
    "Bishop",
    "Blue Blood",
    "Bride",
    "Bridesmaid",
    "British Citizen",
    "Brother",
    "Brother-In-Law",
    "Business Associate",
    "Business Partner",
    "Bystander",
    "Candidate",
    "Cattle Rustler",
    "Client",
    "Co-Worker",
    "Coloratura Soprano",
    "Comedian",
    "Commander In Chief",
    "Commanding Officer",
    "Common-Law Husband",
    "Computer Hacker",
    "Con Artist",
    "Constituent",
    "Copy Cat",
    "Cousin",
    "Crowd-Pleaser",
    "Crybaby",
    "Cub Reporter",
    "Deacon",
    "Dean Of Students",
    "Department Head",
    "Devoted Husband",
    "Diplomat",
    "Doctor Of Philosophy",
    "Double Agent",
    "Drama Student",
    "Drifter",
    "Elder Statesman",
    "England's Queen Mother",
    "Equestrian",
    "Ex-Wife",
    "Executive",
    "Expert Witness",
    "Fair-Weather Friend",
    "Father",
    "First Cousin",
    "Fisherman",
    "Foot Soldier",
    "Football Tackle",
    "Fortune Hunter",
    "Friend Of The Family",
    "Gardener",
    "Genius",
    "Gentleman",
    "Globe Trotter",
    "Good Samaritan",
    "Government Official",
    "Grandfather",
    "Horseback Rider",
    "Host Of A Party",
    "Independent Investor",
    "Insider",
    "Johnny-Come-Lately",
    "Juvenile",
    "Lame Duck President",
    "Landlubber",
    "Landowner",
    "Leader Of The Band",
    "Legal Occupant",
    "Lieutenant Colonel",
    "Light Sleeper",
    "Local Hero",
    "Lounge Singer",
    "Major General",
    "Manager",
    "Matron",
    "Middleman",
    "Minor",
    "Monday Morning Quarterback",
    "Most Likely To Succeed",
    "Mother",
    "Mother Of The Bride",
    "Native American Shaman",
    "Nephew",
    "Niece",
    "Night Owl",
    "Nobel Prize Winner",
    "Officer Of The Court",
    "Opponent",
    "Optimist",
    "Original Owner",
    "Outlaw",
    "Patriot",
    "People-Watcher",
    "Pessimist",
    "Philosopher",
    "Platoon Leader",
    "Platoon Sergeant",
    "Player Of The Year",
    "Pre-Med Student",
    "Priest",
    "Prophet",
    "Proprietor",
    "Protagonist",
    "Publisher",
    "Reader",
    "Redhead",
    "Renaissance Man",
    "Representative",
    "Rodeo Cowboy",
    "Roman Gladiator",
    "Rookie Of The Year",
    "Runner-Up",
    "Scapegoat",
    "Schoolboy",
    "Scoundrel",
    "Screwball",
    "Second Cousin",
    "Shareholder",
    "Shipwrecked Sailor",
    "Shrinking Violet",
    "Significant Other",
    "Silent Partner",
    "Sister",
    "Sister-In-Law",
    "Skin Diver",
    "Sleepyhead",
    "Sob Sister",
    "Soft Touch",
    "Sophomore",
    "Soprano",
    "Sorority Sister",
    "Southern Belle",
    "Spanish Senorita",
    "Sports Fan",
    "Spouse",
    "Staff Member",
    "Staff Sergeant",
    "Star Of Stage And Screen",
    "State Trooper",
    "Stepfather",
    "Stockholder",
    "Storyteller",
    "Student Teacher",
    "Substitute Teacher",
    "Supervisor",
    "Taxpayer",
    "Teaching Fellow",
    "Tenderfoot",
    "Tenor",
    "The Duke Of York",
    "The Prince Of Wales",
    "Ticket Scalper",
    "Tomboy",
    "Tourist",
    "Traitor",
    "Trustee",
    "Tyrant",
    "Umpire",
    "Uncle",
    "Ventriloquist",
    "Very Important Person",
    "Veteran",
    "Violinist",
    "Virtuoso",
    "Weight-Lifter",
    "Young Man"
],
"Before & After": [
    "A Blast From The Past Tense",
    "A Long Shot In The Dark",
    "A Marked Man Of The World",
    "A Touch Of Class Clown",
    "Abraham Lincoln Continental",
    "Adam's Apple Pie",
    "Against All Odds Or Evens",
    "Agatha Christie Brinkley",
    "Alexander The Great Balls Of Fire",
    "All The World's A Stage Fright",
    "American Red Cross Your Heart",
    "An Ear Of Corn Syrup",
    "Baking Soda Fountain",
    "Baton Rouge Louisiana Purchase",
    "Beat The Odds And Ends",
    "Benjamin Franklin D. Roosevelt",
    "Betsy Ross Perot",
    "Billie Jean King Of The Road",
    "Billy Joel Grey",
    "Block That Kick In The Pants",
    "Blow Off A Little Steam Engine",
    "Carrie Fisher-Price",
    "Carrying A Torch Song Trilogy",
    "Central Park Avenue",
    "Charlie Brown Bear",
    "Cleopatra's Barge In On",
    "Coat Of Paint By Numbers",
    "Cost Of Living Well Is The Best Revenge",
    "Cover Girl Scout",
    "Dallas Cowboys And Indians",
    "Debbie Reynolds Wrap",
    "Dolley Madison Square Garden",
    "Don't Fence Me In The Mood",
    "Down In The Valley Forge",
    "Down The Hatch An Egg",
    "Eggs Over Easy Come Easy Go",
    "Emerald Green Bay Packers",
    "Emily Post Office Box",
    "Empty Space Shuttle",
    "Fast Food For Thought",
    "Fat Lip Gloss",
    "Fine Art Carney",
    "Floor Mats Wilander",
    "Flower Power Lunch",
    "Fort Worth Its Weight In Gold",
    "Francis Scott Key West Florida",
    "Free Throw Rug",
    "Gentle Ben Vereen",
    "Get Lost And Found",
    "Glad Rags To Riches",
    "Glory Days Of Our Lives",
    "Golden Gate Bridge Game",
    "Gone With The Wind Tunnel",
    "Gourmet Food For Thought",
    "Grace Kelly Green",
    "Grace Under Fire Alarm",
    "Grover Cleveland Ohio",
    "Hail To The Chief Executive Officer",
    "Harry S Truman Capote",
    "Hedge Your Bet Your Life",
    "Here's Mud In Your Eye Of The Needle",
    "Here's Mud In Your Eye Shadow",
    "Hit Or Miss Saigon",
    "Holy Roman Empire State Building",
    "How Could You Dirty Rat",
    "Hubert Humphrey Bogart",
    "Hundred-Dollar Bill Clinton",
    "I Miss You Dirty Rat",
    "In The Mood Swings",
    "India Ink-Jet Printer",
    "It Boggles The Mind Your Own Business",
    "Jack London England",
    "Japanese Beetle Bailey",
    "Jerry Lewis & Clark",
    "Jesse James Michener",
    "Jodie Foster Parents",
    "Joe Louis Armstrong",
    "John Denver Colorado",
    "John Glenn Close",
    "Johnny Carson City Nevada",
    "Judgment Call Of The Wild",
    "Julius Caesar Salad",
    "Kate Jackson Five",
    "Kelly Green Acres",
    "King Of The Road Warrior",
    "Lag Behind The Eight Ball",
    "Leap Of Faith Hope And Charity",
    "Let There Be Light Bulb",
    "Little Orphan Annie Potts",
    "Little Boy Blue Ribbon",
    "Little House On The Prairie Dog",
    "Lloyd Bridges Of Madison County",
    "Lone Star State Of The Union",
    "Love Me Tender Mercies",
    "Lunch Date Palm Tree",
    "Maid Marian The Librarian",
    "Mail Call Of The Wild",
    "Martha Graham Crackers",
    "Master Key Largo",
    "Meet The Press Your Luck",
    "Melrose Place Your Bets",
    "Mess Kit Carson",
    "Minnie Pearl Harbor",
    "Miss Piggy Bank",
    "Moby Dick Tracy",
    "Money Order In The Court",
    "Murphy Brown University",
    "Nancy Drew Barrymore",
    "No Way To Treat A Lady Bird Johnson",
    "Nolan Ryan O'Neal",
    "On The Make Your Move",
    "Order In The Court Jester",
    "Patrick Henry The Eighth",
    "Piccadilly Circus Clown",
    "Plain Jane Eyre",
    "Porgy And Bess Truman",
    "Port Of Call Waiting",
    "Puget Sound Off",
    "Rain Man Of The World",
    "Ralph Lauren Bacall",
    "Ray Charles In Charge",
    "Rich Little House On The Prairie",
    "Ricki Lake Superior",
    "Ricki Lake Worth Florida",
    "Rise And Shine My Shoes",
    "Rob Roy Rogers",
    "Robert Frost On The Pumpkin",
    "Room At The Top Of The Morning",
    "Saint Francis Scott Key",
    "Sally Field Of Dreams",
    "Sam Houston Oilers",
    "Santa Barbara Walters",
    "Scrub The Floor Show",
    "Searching High And Low Tide",
    "Shelley Long Underwear",
    "Shirley Temple University",
    "Shopping Basket Case",
    "Sinclair Lewis & Clark",
    "Sir Walter Raleigh North Carolina",
    "Sir Walter Scott Joplin",
    "Sitting Pretty Please With Sugar On It",
    "Skating On Thin Ice Cream Cone",
    "Sleeping Beauty And The Beast",
    "Sports Car Wash",
    "Stephen King Kong",
    "Steve Martin Short",
    "Susan B. Anthony Hopkins",
    "Swan Lake Erie",
    "Take Five Easy Pieces",
    "Take It Easy Rider",
    "Tennis Elbow Grease",
    "The Brady Bunch Of Grapes",
    "The British Open Wide",
    "The Right Stuff And Nonsense",
    "Theodore Roosevelt Grier",
    "Thomas Jefferson Davis",
    "Toe The Mark Twain",
    "Tootsie Roll Of Quarters",
    "Tower Of London Bridge",
    "West Point Of View",
    "When You Wish Upon A Star Trek",
    "Whitney Houston Texas"
],
"Place": [
    "Aberdeen Scotland",
    "Adriatic Sea",
    "Aegean Sea",
    "Africa",
    "Albania",
    "Albuquerque New Mexico",
    "Allentown",
    "Amarillo Texas",
    "Amazon Region Of Brazil",
    "Amherst College",
    "Amphitheatre",
    "Amsterdam",
    "Anaheim California",
    "Angola",
    "Anguilla",
    "Appalachia",
    "Arlington Virginia",
    "Armenia",
    "Aruba",
    "Aspen Colorado",
    "Atlanta Georgia",
    "Austria",
    "Babylon",
    "Baghdad",
    "Bakersfield California",
    "Baltimore Maryland",
    "Bangladesh",
    "Bangor Maine",
    "Barbados",
    "Baton Rouge",
    "Bavaria",
    "Beauty Salon",
    "Bedroom",
    "Beijing China",
    "Belgrade",
    "Berkeley California",
    "Bern Switzerland",
    "Bethlehem",
    "Bonn Germany",
    "Borneo",
    "Boston's Fenway Park",
    "Bowling Alley",
    "Brighton England",
    "Brisbane Australia",
    "Bristol England",
    "Brittany France",
    "Broadway",
    "Brown University",
    "Bucharest Romania",
    "Buffalo New York",
    "Bulgaria",
    "Bunker Hill",
    "Burgundy Region Of France",
    "Cairo Egypt",
    "Calcutta India",
    "Cambridge",
    "Camden New Jersey",
    "Cardiff Wales",
    "Castle",
    "Charleston South Carolina",
    "Chicago's Loop",
    "Chile",
    "Chinatown",
    "Classroom",
    "Cleveland Ohio",
    "Clothes Closet",
    "College Dorm",
    "Cologne",
    "Colombia",
    "Columbia University",
    "Copper Mine",
    "Coral Reef",
    "Cornell University",
    "Corvallis Oregon",
    "Country Club",
    "County Seat",
    "Courtroom",
    "Coventry England",
    "Cycle Shop",
    "Danbury Connecticut",
    "Dartmouth College",
    "Dayton Ohio",
    "Daytona Beach Florida",
    "Dearborn Michigan",
    "Decatur Georgia",
    "Denver",
    "Deserted Island",
    "Disneyland",
    "Djakarta",
    "Dominican Republic",
    "Doorway",
    "Down On The Farm",
    "Downtown",
    "Duluth Minnesota",
    "Dungeon",
    "Durham North Carolina",
    "Dusseldorf",
    "Dwelling",
    "East Hampton Long Island",
    "El Salvador",
    "England",
    "Englewood New Jersey",
    "Erie Canal",
    "Estonia",
    "Eugene Oregon",
    "Europe",
    "Evanston Illinois",
    "Fairfield Connecticut",
    "Fairgrounds",
    "Fairway",
    "Family Farm",
    "Filling Station",
    "Florida Everglades",
    "Fordham University",
    "Formosa",
    "Fort Leavenworth",
    "Frankfort Kentucky",
    "Frankfurt Germany",
    "Freeport Bahamas",
    "Fresno California",
    "Gainesville Florida",
    "Gallup New Mexico",
    "Garage",
    "Gas Station",
    "Genoa Italy",
    "Georgetown",
    "Germany",
    "Gettysburg",
    "Gettysburg National Military Park",
    "Ghana",
    "Gibraltar",
    "Glendale California",
    "Gloucester",
    "Grade School",
    "Granada Spain",
    "Grand Cayman",
    "Great Bear Lake Canada",
    "Great Britain",
    "Greece",
    "Green Room",
    "Grenada",
    "Hamburg Germany",
    "Harbor",
    "Harlem",
    "Harrisburg",
    "Hartford Connecticut",
    "Hattiesburg Mississippi",
    "Heidelberg Germany",
    "Hermosa Beach California",
    "Highway Rest Stop",
    "Hiroshima Japan",
    "Hoboken New Jersey",
    "Holland",
    "Hollywood",
    "Hometown",
    "Hospital",
    "Hotel Room",
    "Hotel Suite",
    "Houston Texas",
    "Hungary",
    "Hunting Lodge",
    "Ice Rink",
    "Iceland",
    "India",
    "Indochina",
    "Inverness Scotland",
    "Irvine California",
    "Italy",
    "Ithaca New York",
    "Jamestown Virginia",
    "Jericho",
    "Joe Robbie Stadium In Miami",
    "Joliet Illinois",
    "Junior High School",
    "Kabul Afghanistan",
    "Key Largo Florida",
    "Key West Florida",
    "Khartoum Sudan",
    "Kuwait",
    "Kyoto Japan",
    "Lafayette Louisiana",
    "Lake Huron",
    "Lake Erie",
    "Lakehurst Naval Air Station",
    "Lancaster",
    "Lansing Michigan",
    "Laredo Texas",
    "Leeds England",
    "Library",
    "Libya",
    "London's Hyde Park",
    "London's Mayfair District",
    "Los Angeles",
    "Los Alamos New Mexico",
    "Lucerne Switzerland",
    "Lyon France",
    "Madison Avenue",
    "Madrid Spain",
    "Managua Nicaragua",
    "Manhattan",
    "Manila Philippines",
    "Manitoba Canada",
    "Marin County California",
    "Martha's Vineyard",
    "Mesopotamia",
    "Miami Florida",
    "Michigan's Upper Peninsula",
    "Micronesia",
    "Middletown Connecticut",
    "Milwaukee Wisconsin",
    "Minnesota",
    "Modesto California",
    "Monaco",
    "Montclair New Jersey",
    "Montenegro",
    "Monticello",
    "Montserrat",
    "Morocco",
    "Moscow's Red Square",
    "Mount Holyoke College",
    "Mount Shasta",
    "Movie Theater",
    "Muncie Indiana",
    "Munich Germany",
    "Naples Florida",
    "Nassau",
    "Natchez Mississippi",
    "Nazareth",
    "Never-Never Land",
    "Nevis West Indies",
    "New Jersey",
    "New Orleans Louisiana",
    "New Jersey Turnpike",
    "New York Thruway",
    "New York's Central Park",
    "New York's Harlem",
    "New York's Hudson River",
    "New York's Lake Chautauqua",
    "New York's Lake Onondaga",
    "New York's Westchester County",
    "Norfolk Virginia",
    "Normandy",
    "Norwalk Connecticut",
    "Nursery School",
    "Observation Booth",
    "Ogden Utah",
    "Olympia Washington",
    "Omaha Nebraska",
    "Orlando Florida",
    "Oshkosh Wisconsin",
    "Oxford Mississippi",
    "Oxnard California",
    "Palermo Sicily",
    "Palestine",
    "Palm Beach",
    "Palm Springs",
    "Paris France",
    "Park Place",
    "Patagonia",
    "Peninsula",
    "Pennsylvania Dutch Country",
    "Penthouse",
    "Peoria Illinois",
    "Permanent Residence",
    "Perth Australia",
    "Phone Booth",
    "Pittsburgh",
    "Plainfield New Jersey",
    "Pompeii",
    "Port Of Call",
    "Portsmouth",
    "Power Plant",
    "Prague",
    "Prep School",
    "Prescott Arizona",
    "Princeton University",
    "Prison Cell",
    "Promenade Deck",
    "Proving Ground",
    "Prussia",
    "Public Library",
    "Puerto Rico",
    "Purdue University",
    "Quantico Virginia",
    "Quebec City",
    "Queensland Australia",
    "Quincy Illinois",
    "Racetrack",
    "Redmond Washington",
    "Reform School",
    "Residence",
    "Restroom",
    "Rhodes Greece",
    "Rimini Italy",
    "Rochester New York",
    "Rock Garden",
    "Rockland County New York",
    "Rome's Tiber River",
    "Rooming House",
    "Root Cellar",
    "Rutgers University",
    "Rutland Vermont",
    "Safe Haven",
    "Saint Petersburg Florida",
    "Salad Bar",
    "Saloon",
    "San Francisco",
    "San Francisco's Haight-Ashbury",
    "Sand Trap",
    "Santa Barbara California",
    "Sarasota Florida",
    "Saratoga New York",
    "Savannah Georgia",
    "Scandinavia",
    "Scarsdale New York",
    "Scene Of The Crime",
    "School House",
    "Schoolroom",
    "Seaport",
    "Seat Of Government",
    "Seattle Washington",
    "Selma Alabama",
    "Seneca Falls New York",
    "Senior High School",
    "Service Station",
    "Seville Spain",
    "Shiloh Tennessee",
    "Shoe Repair Shop",
    "Siena Italy",
    "Singapore",
    "Ski Resort",
    "Snake Pit",
    "Solomon Islands",
    "Sonoma County California",
    "Sonora Mexico",
    "Soup Kitchen",
    "South Dakota's Badlands",
    "South Korea",
    "Southampton New York",
    "Sparta Greece",
    "St. Petersburg Russia",
    "St. Thomas",
    "St. Croix",
    "Stable",
    "Stadium",
    "Stairwell",
    "Staten Island",
    "Stateroom",
    "Steamboat Springs Colorado",
    "Stockton California",
    "Store",
    "Street Corner",
    "Stronghold",
    "Study Hall",
    "Subdivision",
    "Sudan",
    "Suez Canal",
    "Sumatra Indonesia",
    "Summer School",
    "Summer Camp",
    "Sunnyvale California",
    "Switzerland",
    "Sydney Australia",
    "Tahiti",
    "Tallahassee Florida",
    "Tampa Florida",
    "Taos New Mexico",
    "Tarrytown New York",
    "Tavern",
    "Tempe Arizona",
    "Terrace",
    "The Adirondacks",
    "The African Nation Of Chad",
    "The Allegheny Mountains",
    "The Atlantic Ocean",
    "The Australian Outback",
    "The Canadian Province Of Alberta",
    "The Catskill Mountains",
    "The Country Of Laos",
    "The Cumberland Gap",
    "The District Of Columbia",
    "The French Riviera",
    "The Garden Of Eden",
    "The Himalayas",
    "The Island Of Malta",
    "The Italian Riviera",
    "The Library Of Congress",
    "The Midwest",
    "The Mississippi Delta",
    "The Netherlands",
    "The Planet Mars",
    "The Planet Neptune",
    "The Planet Saturn",
    "The Planet Jupiter",
    "The Planet Uranus",
    "The Poconos",
    "The Potomac River",
    "The Rhine River",
    "The River Nile",
    "The Rockies",
    "The Sahara Desert",
    "The Tigris River",
    "The Wabash River",
    "The West Indies",
    "The Yucatan Peninsula",
    "The Yukon",
    "The Alps",
    "The Andes Mountains",
    "The Bronx",
    "The Czech Republic",
    "The French Quarter In New Orleans",
    "The Island Of Bali",
    "The Island Of Guam",
    "The Isle Of Wight",
    "The Lost Continent Of Atlantis",
    "The Mason-Dixon Line",
    "The North Pole",
    "The North Pole",
    "The Orange Bowl In Miami",
    "The Ozarks",
    "The Rim Of The Grand Canyon",
    "The Rose Bowl In Pasadena",
    "The Sea Of Galilee",
    "The Set Of A Movie",
    "The South American Country Of Peru",
    "Toledo Ohio",
    "Topeka Kansas",
    "Toronto Ontario Canada",
    "Tourist Trap",
    "Town",
    "Town Hall",
    "Townhouse",
    "Train Station",
    "Tropic Of Cancer",
    "Tropic Of Capricorn",
    "Tucson Arizona",
    "Tulane University",
    "Tunisia",
    "Tuscaloosa Alabama",
    "Tuscany",
    "Uganda",
    "Ukraine",
    "United States Naval Academy",
    "United States Of America",
    "Used Car Lot",
    "Utah's Bonneville Salt Flats",
    "Utica New York",
    "Vail Colorado",
    "Valhalla",
    "Valley Forge",
    "Vancouver British Columbia",
    "Vassar College",
    "Vicksburg Mississippi",
    "Vietnam",
    "Volcano's Crater",
    "Walden Pond",
    "Warehouse",
    "Waterbury Connecticut",
    "West Point",
    "Windsor Ontario",
    "Winner's Circle",
    "Workroom",
    "Yakima Washington",
    "Yale University",
    "Yokohama Japan",
    "Yonkers New York",
    "Yorktown Virginia",
    "Youngstown Ohio",
    "Yuma Arizona",
    "Zermatt Switzerland",
    "Zimbabwe"
],
"Landmark": [
    "Acadia National Park",
    "Bermuda's Pink Sand Beaches",
    "Big Bend National Park",
    "Boston Common",
    "Bryce Canyon National Park",
    "Buckingham Palace",
    "Death Valley National Monument",
    "Edison's Home In Menlo Park New Jersey",
    "Ellis Island",
    "Fort Davis National Historic Site",
    "Fort Point National Historic Site",
    "Fort Smith National Historic Site",
    "Glacier National Park",
    "Grand Canyon National Park",
    "Hyde Park National Historic Site",
    "Independence Hall In Philadelphia",
    "India's Ganges River",
    "Jerusalem's Wailing Wall",
    "John Muir National Historic Site",
    "Kings Canyon National Park",
    "Lassen Volcanic National Park",
    "London Bridge",
    "London's Covent Garden",
    "London's Tower Bridge",
    "Mesa Verde National Park",
    "Mount Palomar Observatory",
    "Mount Rushmore",
    "Muir Woods National Monument",
    "Niagara Falls",
    "Panama Canal",
    "Petrified Forest National Park",
    "Piccadilly Circus",
    "Plymouth Rock",
    "Redwood National Park",
    "San Francisco's Fisherman's Wharf",
    "Scotland Yard",
    "Sequoia National Park",
    "Serengeti National Park",
    "Signal Hill National Historic Site",
    "South Carolina's Fort Sumter",
    "Stonehenge",
    "The Eiffel Tower",
    "The Jefferson Memorial",
    "The Lincoln Memorial",
    "The Palace Of Versailles",
    "The Parthenon",
    "The Smithsonian Institution",
    "The Waldorf-Astoria Hotel",
    "The Washington Monument",
    "The White House",
    "The Alamo",
    "The Boston Post Road",
    "The Egyptian Pyramids",
    "The Empire State Building",
    "The Gateway Arch In St. Louis",
    "The Golden Gate Bridge",
    "The Great Wall Of China",
    "The Kremlin In Moscow",
    "The Mission Of San Juan Capistrano",
    "The Rock Of Gibraltar",
    "The Sphinx",
    "The Tower Of London",
    "Thomas Jefferson's Monticello",
    "Tokyo's Ginza District",
    "Valley Forge National Historical Park",
    "Walden Pond",
    "Westminster Abbey",
    "Winchester Cathedral",
    "Windsor Castle"
],
"Fictional Characters": [
    "Beavis & Butthead",
    "Big Bird Bert & Ernie",
    "Gumby & Pokey",
    "Hansel & Gretel",
    "Jonny Quest & Bandit",
    "Mermaids",
    "Mufasa Simba & Scar",
    "Ren & Stimpy",
    "Rocky & Bullwinkle",
    "The Dynamic Duo",
    "The Hardy Boys",
    "The Simpsons",
    "Tom And Jerry"
],
"Family": [
    "Blythe Danner & Gwyneth Paltrow",
    "Bruce & Laura Dern",
    "Judy Garland & Liza Minelli",
    "Loretta Lynn & Crystal Gayle",
    "Michael & Olympia Dukakis",
    "Naomi Wynonna & Ashley Judd",
    "Tony & Jamie Lee Curtis",
    "Vanessa & Lynn Redgrave",
    "Warren Beatty & Shirley Maclaine",
    "William Stephen & Alec Baldwin"
],
"Thing": [
    "A Glass Of Chablis",
    "A Kind Word",
    "A Lump Of Coal In Your Stocking",
    "Abbreviation",
    "Accident",
    "Accomplishment",
    "Achievement Award",
    "Acorn",
    "Acupuncture",
    "Adhesive Tape",
    "Adventure Story",
    "Adversity",
    "Advertising Agency",
    "Afghan Hound",
    "After-Dinner Drink",
    "Afternoon Snack",
    "Airplane",
    "Alarm Clock",
    "Algebra Textbook",
    "Allergy Shot",
    "Amber",
    "Amendment",
    "An Even Tan",
    "An Hour",
    "Anatomy",
    "Ancient History",
    "Anteater",
    "Antibiotic",
    "Apology",
    "Apple",
    "Appointment",
    "Aquamarine",
    "Aquarium",
    "Arrow",
    "Arrowhead",
    "Atmosphere",
    "Avocado",
    "Award",
    "Baby Face",
    "Baby Talk",
    "Baby's Crib",
    "Backpack",
    "Backup Generator",
    "Badger",
    "Bald Eagle",
    "Ball-Point Pen",
    "Ballad",
    "Balloon",
    "Bamboo",
    "Banana",
    "Banana Peel",
    "Barrage Of Questions",
    "Baseball Bat",
    "Baseball Cap",
    "Basket",
    "Basset Hound",
    "Bathroom Scale",
    "Bathtub",
    "Baton",
    "Battery",
    "Bay Window",
    "Beacon",
    "Beagle",
    "Bedspread",
    "Biography",
    "Birch Tree",
    "Birthmark",
    "Birthright",
    "Black Belt In Karate",
    "Black Bear",
    "Black Hole In Outer Space",
    "Blackberry",
    "Blackbird",
    "Blender",
    "Blood Type",
    "Bloodhound",
    "Blossom",
    "Blue Chip Stock",
    "Blueprint",
    "Board Game",
    "Book Review",
    "Boot Polish",
    "Bottomless Pit",
    "Box Score",
    "Boxcar",
    "Bracelet",
    "Braille",
    "Bread Machine",
    "Briefcase",
    "Brontosaurus",
    "Brush Stroke",
    "Bubble Bath",
    "Bulldog",
    "Bullet",
    "Bushel Basket",
    "Butter Churn",
    "Button",
    "Buttonhole",
    "Calculator",
    "Calendar",
    "Can Opener",
    "Can Of Soda",
    "Canceled Stamp",
    "Candlestick",
    "Candy Cane",
    "Canoe Paddle",
    "Canteen",
    "Cappuccino Maker",
    "Care Package",
    "Carrot Cake",
    "Catalog",
    "Catamaran",
    "Catapult",
    "Category",
    "Cathedral",
    "Cauliflower",
    "Cedar",
    "Ceiling Fan",
    "Cellular Phone",
    "Centimeter",
    "Chalk",
    "Cheese",
    "Cheetah",
    "Chemical Formula",
    "Chessboard",
    "Chicken",
    "Chicken Fried Steak",
    "Chimpanzee",
    "Chipmunk",
    "Chord",
    "Chorus",
    "Christmas Bonus",
    "Church Bell",
    "Clam Chowder",
    "Cleanser",
    "Closet Space",
    "Cloverleaf",
    "Coat And Tie",
    "Cocker Spaniel",
    "Coffee Cake",
    "Coincidence",
    "Coincidence",
    "College Scholarship",
    "Community",
    "Companionship",
    "Computer",
    "Confidence",
    "Conifer Tree",
    "Constellation",
    "Contemporary Jazz",
    "Controversy",
    "Cookie Cutter",
    "Corkscrew",
    "Corn Bread",
    "Cornucopia",
    "Cottage",
    "Counter",
    "Court Case",
    "Cover Charge",
    "Cranberry",
    "Crane",
    "Cypress Tree",
    "Dachshund",
    "Dalmatian",
    "Dear John Letter",
    "Dial Tone",
    "Dictionary Definition",
    "Dijon Mustard",
    "Dinosaur",
    "Diplomatic Immunity",
    "Dirty Trick",
    "Doberman Pinscher",
    "Dolphin",
    "Doorstep",
    "Doorstop",
    "Dot Matrix Printer",
    "Double Negative",
    "Double-Decker Bus",
    "Douglas Fir",
    "Dow Jones Industrial Average",
    "Dragon",
    "Dripping Faucet",
    "Drive-In Movie",
    "Duet",
    "Duffel Bag",
    "Dump Truck",
    "Dutch Elm Disease",
    "Earthworm",
    "Easter Lily",
    "Eggplant Parmigiana",
    "Einstein's Theory Of Relativity",
    "Electric Blanket",
    "Electricity",
    "Element",
    "Elephant",
    "Elm Tree",
    "Empty Nest",
    "End Table",
    "English Translation",
    "Entertainment",
    "Envelope",
    "Escape Hatch",
    "Estimate",
    "Etching",
    "Evergreen Tree",
    "Evidence",
    "Expense Account",
    "Express Lane",
    "Extension Cord",
    "Extension Ladder",
    "Extra-Large Size",
    "Falcon",
    "False Alarm",
    "Family Heirloom",
    "Family Name",
    "Felt Tip Pen",
    "Filter",
    "Final Adjustment",
    "Finch",
    "Finger Food",
    "Finger Paint",
    "Fingernail",
    "Finish Line",
    "Fire Hose",
    "Fire Insurance",
    "Fireplace",
    "Firewood",
    "First Day Of The Month",
    "First-Class Mail",
    "Five-Digit Zip Code",
    "Flashlight",
    "Flat Tire",
    "Flavor Of The Month",
    "Flood Insurance",
    "Floor Plan",
    "Floppy Disk",
    "Flower Arrangement",
    "Flu Vaccination",
    "Flu Shot",
    "Fluorescent Light",
    "Flying Fish",
    "Food Processor",
    "Footrest",
    "Form Letter",
    "Fountain Of Youth",
    "Fountain Pen",
    "Fox Terrier",
    "Free Enterprise",
    "Free Offer",
    "French Bread",
    "French Horn",
    "French Poodle",
    "Frisbee",
    "Fruit Punch",
    "Funnel",
    "Gall Bladder",
    "Gallon",
    "Gander",
    "Garden Hose",
    "Gardenia",
    "General Assessment",
    "Geometry",
    "Get-Rich-Quick Scheme",
    "Giggle",
    "Gingerbread",
    "Goat Cheese",
    "Golden Retriever",
    "Good Advice",
    "Good Penmanship",
    "Grab Bag",
    "Grape",
    "Graph Paper",
    "Gravel",
    "Grease Paint",
    "Great White Shark",
    "Great Dane",
    "Greek Food",
    "Green Grass",
    "Greyhound",
    "Grizzly Bear",
    "Ground Beef",
    "Groundhog",
    "Group Rate",
    "Growth Rate",
    "Guide Dog",
    "Guilty Conscience",
    "Gutter",
    "Haircut",
    "Hairstyle",
    "Half An Hour",
    "Ham And Cheese Sandwich",
    "Handbook",
    "Handlebar Mustache",
    "Handwriting",
    "Happiness",
    "Help Wanted Ad",
    "Hemlock",
    "Hickory",
    "High Fever",
    "High School Diploma",
    "High-Fiber Cereal",
    "High-Five",
    "Higher Education",
    "Hippopotamus",
    "Hiring Freeze",
    "Holy Roman Empire",
    "Homesickness",
    "Homework Assignment",
    "Honeybee",
    "Hook Shot",
    "Hoot Owl",
    "Horizon",
    "Hornet's Nest",
    "Horoscope",
    "Horseradish",
    "Hot Compress",
    "Hot Sauce",
    "Hourglass",
    "Houseboat",
    "Huckleberry",
    "Human Nature",
    "Humility",
    "Ice Pick",
    "Income Tax",
    "Indelible Ink",
    "Initiative",
    "Inner Tube",
    "Invisible Ink",
    "Irish Setter",
    "Jade Pendant",
    "Jaguar",
    "Javelin",
    "Jawbone",
    "Jazz Music",
    "Jelly Doughnut",
    "Journal",
    "Journalism",
    "Keepsake",
    "Kentucky Bluegrass",
    "Keynote Address",
    "Kickstand",
    "Kid Gloves",
    "Kilogram",
    "Knee Jerk Reaction",
    "Knockout Blow",
    "Knot",
    "Knotty Pine",
    "Laptop Computer",
    "Lasagna",
    "Laser Printer",
    "Lasso",
    "Laughter",
    "Laundry Cart",
    "Lawn Mower",
    "Leaf Blower",
    "Learner's Permit",
    "Leopard",
    "Lettuce",
    "Lhasa Apso",
    "Liar's Poker",
    "Life Preserver",
    "Lifeboat",
    "Light",
    "Lighthouse",
    "Lightning",
    "Limestone",
    "Liter",
    "Lithograph",
    "Litmus Test",
    "Little Black Book",
    "Lobster",
    "Lock Of Hair",
    "Loophole",
    "Loudspeaker",
    "Lovebird",
    "Low-Cholesterol Diet",
    "Low-Fat Diet",
    "Macaw",
    "Magic Carpet",
    "Magic Marker",
    "Mallard",
    "Mandarin Orange",
    "Mango",
    "Mangrove",
    "Mannequin",
    "Map Of The World",
    "Maple Tree",
    "Marmalade",
    "Marriage",
    "Marshmallow",
    "Masterpiece",
    "Matchbook",
    "Maternal Instinct",
    "Meadow",
    "Melody",
    "Melon",
    "Melted Cheese",
    "Memoir",
    "Memorandum",
    "Memory",
    "Metaphor",
    "Meter",
    "Mexican Sombrero",
    "Microphone",
    "Microscope",
    "Microwave Oven",
    "Mint Julep",
    "Minus Sign",
    "Minute",
    "Mixed Blessing",
    "Mobile Phone",
    "Monday Morning Traffic",
    "Monkey",
    "Monument",
    "Morning Glory",
    "Motorcycle",
    "Mountain Bike",
    "Mountain Goat",
    "Mountain Lion",
    "Mountain Range",
    "Multiple-Choice Question",
    "Mushroom",
    "Mustard",
    "Mutual Understanding",
    "Navy Blue Suit",
    "Necklace",
    "Negative Feedback",
    "Neon Sign",
    "Net Profit",
    "Newspaper",
    "Night Table",
    "Nightgown",
    "Nightingale",
    "Noisemaker",
    "Nonstop Flight",
    "Nourishment",
    "Nuclear Physics",
    "Nuclear Power Plant",
    "Oak Tree",
    "Oatmeal",
    "Obligation",
    "Octopus",
    "Oil Well",
    "Old English Sheepdog",
    "Olive Oil",
    "Opera",
    "Opinion",
    "Orange",
    "Orange Blossom",
    "Orange Tree",
    "Ornament",
    "Ounce",
    "Over-The-Counter Medication",
    "Pair Of Pants",
    "Palm Tree",
    "Pamphlet",
    "Panda",
    "Pane Of Glass",
    "Papaya",
    "Paragraph",
    "Park Bench",
    "Parking Meter",
    "Parrot",
    "Pause",
    "Pay-Per-View Television",
    "Peace Offering",
    "Peach",
    "Peach Cobbler",
    "Pear Tree",
    "Peat Moss",
    "Pecan",
    "Peck Of Pickled Peppers",
    "Pedestal",
    "Peer Pressure",
    "Penguin",
    "Percolator",
    "Perfect Pitch",
    "Periscope",
    "Phone Call",
    "Piece Of Pie",
    "Pigeon",
    "Pillow",
    "Pilot Light",
    "Pine Tree",
    "Pint Of Milk",
    "Pipe Dream",
    "Plastic Surgery",
    "Pocket",
    "Pocket Comb",
    "Poetic Justice",
    "Poetic License",
    "Polar Ice Cap",
    "Pollution",
    "Popcorn",
    "Pork Chop",
    "Porpoise",
    "Porterhouse Steak",
    "Porthole",
    "Postponement",
    "Pound",
    "Prayer Book",
    "Press Pass",
    "Pressure Cooker",
    "Prime Time Television",
    "Printing Press",
    "Prompt Response",
    "Proper Noun",
    "Prosperity",
    "Public Address System",
    "Pulitzer Prize",
    "Pulley",
    "Pup Tent",
    "Puppet Show",
    "Purchase Order",
    "Putting Green",
    "Pyramid",
    "Python",
    "Quantum Physics",
    "Quart",
    "Quartz Watch",
    "Rabbit",
    "Racehorse",
    "Radio Broadcast",
    "Railroad",
    "Rainbow",
    "Rap Music",
    "Raspberry",
    "Ratings System",
    "Rattlesnake",
    "Raven",
    "Ream Of Paper",
    "Receding Hairline",
    "Recipe",
    "Recliner",
    "Record Album",
    "Rectangle",
    "Red Herring",
    "Refrigerator",
    "Relationship",
    "Remedy",
    "Reminder",
    "Rent Payment",
    "Retirement Age",
    "Rhinoceros",
    "Rhythm",
    "Rice Paper",
    "Ripple In The Water",
    "Road Map",
    "Roadblock",
    "Robot",
    "Rock Cornish Game Hen",
    "Rocking Chair",
    "Rocky Road Ice Cream",
    "Rolling Pin",
    "Room Service",
    "Rooster",
    "Rose Bush",
    "Rosebud",
    "Rubber",
    "Rubber Band",
    "Rubber Stamp",
    "Rush Hour Traffic",
    "Rust Remover",
    "Rye Bread",
    "Sacrifice Fly",
    "Saddle",
    "Safety Net",
    "Saffron",
    "Sagebrush",
    "Saint Bernard Dog",
    "Salad Dressing",
    "Salary",
    "Saltwater Fish",
    "Sand Bag",
    "Sand Dollar",
    "Sandpaper",
    "Sandstone",
    "Sandwich",
    "Sapphire",
    "Satin",
    "Sausage",
    "Savings And Loan Association",
    "Savings Bond",
    "Schnauzer",
    "Scholarship",
    "School Play",
    "School Year",
    "Score Card",
    "Scotch Tape",
    "Screen Saver",
    "Screen Test",
    "Sea Urchin",
    "Sea Horse",
    "Sea Lion",
    "Seafood",
    "Seagull",
    "Search Warrant",
    "Searchlight",
    "Secret Identity",
    "Seeing Eye Dog",
    "Self-Control",
    "Self-Esteem",
    "Self-Portrait",
    "Semi-Circle",
    "Semicolon",
    "Sense Of Smell",
    "Sentence",
    "Septic Tank",
    "Sequoia",
    "Service Elevator",
    "Sewing Machine",
    "Shadow",
    "Shag Carpet",
    "Shark Fin",
    "Sheet Music",
    "Shellfish",
    "Shipment",
    "Shoehorn",
    "Shooting Star",
    "Shopping Cart",
    "Short Story",
    "Shorthand",
    "Shoulder",
    "Shovel",
    "Show Business",
    "Shower",
    "Shrink-Wrap",
    "Side Street",
    "Sign Language",
    "Silence",
    "Silkworm",
    "Silver Platter",
    "Skeleton",
    "Ski Lift",
    "Skim Milk",
    "Skylight",
    "Sleeping Bag",
    "Slingshot",
    "Slow Dance",
    "Smoke Screen",
    "Snapdragon",
    "Snapshot",
    "Snowball",
    "Snowdrift",
    "Soapbox",
    "Social Security Check",
    "Soda Fountain",
    "Soda Water",
    "Sofa Bed",
    "Software",
    "Solar System",
    "Song And Dance Routine",
    "Sonic Boom",
    "Sore Throat",
    "Soup Ladle",
    "Southbound Train",
    "Souvenir",
    "Space Heater",
    "Spaghetti Sauce",
    "Spare Change",
    "Spark Plug",
    "Sparkling Mineral Water",
    "Sparrow",
    "Spearmint Gum",
    "Special Assignment",
    "Specimen",
    "Spectacle",
    "Speedboat",
    "Spell-Checker",
    "Spider Monkey",
    "Spinal Column",
    "Spine-Tingling Adventure",
    "Spinning Wheel",
    "Splinter",
    "Split Decision",
    "Split-Level House",
    "Spruce",
    "Stack Of Paper",
    "Stallion",
    "Stamp",
    "Starfish",
    "Starry Night",
    "Status Report",
    "Steam Engine",
    "Steel Wool",
    "Steep Hill",
    "Stegosaurus",
    "Stencil",
    "Step Stool",
    "Stepstool",
    "Sterling Silver",
    "Stick",
    "Stiff Neck",
    "Stingray",
    "Stop Sign",
    "Stove",
    "Stovepipe Hat",
    "Strand Of Hair",
    "Strawberry Jam",
    "Stretcher",
    "String Bean",
    "Striped Bass",
    "Stumbling Block",
    "Submarine",
    "Sunny Afternoon",
    "Sunshine",
    "Sunstroke",
    "Survey",
    "Sweat Suit",
    "Sweater",
    "Swimming Pool",
    "Swiss Bank Account",
    "Swordfish",
    "Sycamore Tree",
    "Tabasco Sauce",
    "Table Tennis",
    "Tablecloth",
    "Tablespoon",
    "Tail Pipe",
    "Taillight",
    "Talk Show",
    "Tank Top",
    "Tea Service",
    "Technology",
    "Telephone",
    "Telescope",
    "Television Remote Control",
    "Temperature",
    "Test Tube",
    "Thank-You Note",
    "The Bronze Age",
    "The Milky Way",
    "The Mother Lode",
    "The Polka",
    "The Scales Of Justice",
    "The Stone Age",
    "The Fifth Amendment",
    "The First Commandment",
    "The Moon",
    "The Old Testament",
    "Theater In The Round",
    "Thimble",
    "Third-Class Mail",
    "Thirst",
    "Thousand Island Dressing",
    "Throat",
    "Throw Rug",
    "Thunderbolt",
    "Ticket",
    "Timber Wolf",
    "Title Page",
    "Toaster Oven",
    "Tollbooth",
    "Tomato",
    "Toothbrush",
    "Tortoise",
    "Toss-Up Question",
    "Tote Bag",
    "Tour Of Duty",
    "Toy Chest",
    "Trade Agreement",
    "Trademark",
    "Tradition",
    "Transcript",
    "Trap Door",
    "Trapeze",
    "Trash Can",
    "Traveler's Check",
    "Treasure Trove",
    "Triceratops",
    "Trilogy",
    "Tripod",
    "Tulip",
    "Turnstile",
    "Tweed Suit",
    "Twenty-Twenty Vision",
    "Two-By-Four",
    "Typing Paper",
    "Tyrannosaurus Rex",
    "Underpass",
    "Vacuum Cleaner",
    "Violet",
    "VIP Treatment",
    "Vocabulary",
    "Volleyball",
    "Volume Control",
    "Vote Of Confidence",
    "Vow Of Silence",
    "Vulture",
    "Wading Pool",
    "Waffle Iron",
    "Wagon Train",
    "Waistline",
    "Walnut",
    "Walnut",
    "Walrus",
    "Wardrobe",
    "Warp Speed",
    "Watch",
    "Water",
    "Water Balloon",
    "Water Cooler",
    "Water Pump",
    "Waterfall",
    "Waterfowl",
    "Wedding Cake",
    "Weekly Paycheck",
    "Weeping Willow Tree",
    "Wheat Germ",
    "Whiplash",
    "Wide-Angle Lens",
    "Wild Rice",
    "Will Power",
    "Willow",
    "Window Shade",
    "Winter Squash",
    "Wisdom",
    "Wisdom Tooth",
    "Woodpile",
    "Worcestershire Sauce",
    "World Atlas",
    "Wristband",
    "Wristwatch",
    "Writer's Cramp",
    "Yellow-Bellied Sapsucker"
],
"People": [
    "A Foursome For Golf",
    "Abbott And Costello",
    "Absentee Voters",
    "Ace Of Base",
    "Actors",
    "Amish Farmers",
    "Atlanta Falcons",
    "Australians",
    "Baby Boomers",
    "Baritone Bass & Tenor",
    "Beat Poets",
    "Bebe & Cece Winans",
    "Blood Relatives",
    "Boy Scout Troop",
    "Britain's Labor Party",
    "Cast Members",
    "Celebrity Panel",
    "Celts Picts & Saxons",
    "Chicago Bears",
    "Children",
    "Choir",
    "Coast Guard",
    "Committee Members",
    "Comparison Shoppers",
    "Cops And Robbers",
    "Counting Crows",
    "Crash Test Dummies",
    "Critics",
    "Cypress Hill",
    "Daughters Of The American Revolution",
    "Democrats",
    "Denver Broncos",
    "Distinguished Colleagues",
    "English Majors",
    "Explorers",
    "Family Members",
    "Farm Team",
    "Fast Friends",
    "Father And Son",
    "Firing Squad",
    "Forefathers",
    "Future Generations",
    "Game Show Audience",
    "Glee Club",
    "Grammy Award Winners",
    "Green Day",
    "Grown-Ups",
    "Gypsies",
    "Historical Figures",
    "Hootie And The Blowfish",
    "Husband And Wife",
    "Immigrants",
    "Impostors",
    "Indigo Girls",
    "Infants",
    "Inhabitants",
    "Instructors",
    "Journalists",
    "Jugglers",
    "Justices Of The Supreme Court",
    "Kansas City Chiefs",
    "Kindergarten Students",
    "Kings And Queens",
    "Lads And Lasses",
    "Legislators",
    "Liberals",
    "Lords And Ladies",
    "Los Angeles Raiders",
    "Loyal Customers",
    "Magazine Subscribers",
    "Men Women And Children",
    "Middle-Class Family",
    "Minor League Players",
    "Mohawk Indians",
    "Mother And Daughter",
    "Neighbors",
    "New Orleans Saints",
    "New York Jets",
    "Nine Inch Nails",
    "Nirvana",
    "Old Friends",
    "Opera Singers",
    "Parent-Teacher Association",
    "Parents",
    "Partners In Crime",
    "Pearl Jam",
    "Pen Pals",
    "Physicians",
    "Pink Floyd",
    "Playmates",
    "Police Officers",
    "Private Investigators",
    "Producer And Director",
    "Proprietors",
    "Republicans",
    "Role Models",
    "Roman Emperors",
    "Roommates",
    "Salt-N-Pepa",
    "San Francisco Forty-Niners",
    "San Diego Chargers",
    "Search Party",
    "Senior Citizens",
    "Seventh Grade Teachers",
    "Simply Red",
    "Sisters",
    "Small Children",
    "Smashing Pumpkins",
    "Snake Charmers",
    "Soundgarden",
    "Sponsors",
    "Squadron",
    "State Legislature",
    "Sumo Wrestlers",
    "Sunday School Teachers",
    "Switch Hitters",
    "Symphony Orchestra",
    "Talking Heads",
    "Team Members",
    "Teenagers",
    "Texans",
    "The Confederate Army",
    "The Cranberries",
    "The Czars Of Russia",
    "The Doors",
    "The Dream Team",
    "The League Of Nations",
    "The Mighty Ducks",
    "The Monkees",
    "The Peace Corps",
    "The Sheriff & His Posse",
    "The Brat Pack",
    "The Green Berets",
    "The House Ways And Means Committee",
    "The Mighty Ducks",
    "The People Next Door",
    "The Tudor Kings Of England",
    "Toad The Wet Sprocket",
    "Tony Orlando And Dawn",
    "Trade Union",
    "Twins",
    "U.S. Navy Seals",
    "Vikings",
    "Winners And Losers"
],
"Phrase": [
    "A Momentary Lapse",
    "A Reflection Of Our Times",
    "A Cursory Glance",
    "A Dead Ringer",
    "A Far Far Better Thing",
    "A Figment Of Your Imagination",
    "A Fox In The Henhouse",
    "A Fresh Point Of View",
    "A Friend In Need Is A Friend Indeed",
    "A Gleam In Your Eye",
    "A Gutsy Move",
    "A Land-Office Business",
    "A Matter Of Policy",
    "A Meal In Itself",
    "A Moot Point",
    "A Novel Idea",
    "A Penny For Your Thoughts",
    "A Plan Of Action",
    "A Rolling Stone Gathers No Moss",
    "A Run For Your Money",
    "A Slight Chance Of Showers",
    "A Stab In The Back",
    "A Stitch In Time Saves Nine",
    "A Straw In The Wind",
    "A Sure Thing",
    "A Total Loss",
    "Alive And Kicking",
    "All Along The Line",
    "All That Glitters Is Not Gold",
    "All's Fair In Love And War",
    "An Apple A Day Keeps The Doctor Away",
    "An Exception To The Rule",
    "An Extra Pair Of Eyes",
    "An Uphill Battle",
    "And All That Jazz",
    "Animal Vegetable Or Mineral",
    "Are You Decent",
    "Are You Waiting For An Engraved Invitation",
    "Around The Bend",
    "As Pure As The Driven Snow",
    "As Rare As Hens' Teeth",
    "At Any Rate",
    "At The Helm",
    "At The Present Time",
    "At The Same Time",
    "Batteries Not Included",
    "Battling The Forces Of Evil",
    "Be Still",
    "Been Down So Long It Looks Like Up To Me",
    "Before The Parade Passes You By",
    "Beggars Can't Be Choosers",
    "Behave Yourself",
    "Behind The Scenes",
    "Behind The Eight-Ball",
    "Behind The Wheel",
    "Besieged With Requests",
    "Betwixt And Between",
    "Birds Of A Feather Flock Together",
    "Bite The Bullet",
    "Blood Is Thicker Than Water",
    "Brace Yourself",
    "Breath-Taking Beauty",
    "Bring Them Into The Fold",
    "Busy As A Bee",
    "Buying On Margin",
    "By The Nape Of The Neck",
    "Call A Cab",
    "Can A Leopard Change Its Spots",
    "Can I Quote You On That",
    "Cash And Carry",
    "Cash On The Barrelhead",
    "Cast Aspersions",
    "Catch-As-Catch-Can",
    "Caught Short-Handed",
    "Cease And Desist",
    "Change Your Tune",
    "Charity Begins At Home",
    "Check Please",
    "Check Your Sources",
    "Chewing Its Cud",
    "Class Dismissed",
    "Clean Your Room",
    "Clip Coupons",
    "Close Quarters",
    "Closed Until Further Notice",
    "Cross Your Palm With Silver",
    "Cry Yourself To Sleep",
    "Curiosity Killed The Cat",
    "Current Whereabouts Unknown",
    "Cut And Dried",
    "Cut And Paste",
    "Cute As A Button",
    "Dim The Lights",
    "Dinner Is In The Oven",
    "Do Your Duty",
    "Don't Look Back",
    "Don't Make Waves",
    "Don't Talk Back",
    "Don't Be So Naive",
    "Don't Be Such A Pest",
    "Don't Get Nasty With Me",
    "Don't Give Me Any Lip",
    "Don't Just Stand There",
    "Don't Let It Slip Away",
    "Don't Lie Down On The Job",
    "Don't Put All Your Eggs In One Basket",
    "Don't Stand On Ceremony",
    "Don't Take It Lying Down",
    "Double Your Pleasure",
    "Down The Drain",
    "Down Under",
    "Duck For Cover",
    "Earn Your Keep",
    "Easier Said Than Done",
    "East Is East And West Is West",
    "End Of The Line",
    "End Of The Road",
    "Enjoy It While You Can",
    "Enough Is Enough",
    "Every Cloud Has A Silver Lining",
    "Every Tom Dick And Harry",
    "Face To Face",
    "Facing The Future",
    "Fade Away",
    "Fair To Middling",
    "Faith And Trust",
    "Fame And Fortune",
    "Familiarity Breeds Contempt",
    "Famous Last Words",
    "Feed A Cold And Starve A Fever",
    "Feel The Heat",
    "Fifteen Minutes Of Fame",
    "Fighting Tooth And Nail",
    "Fill Me In On It",
    "Find Favor With",
    "Finders Keepers",
    "Fleet Of Foot",
    "Fluent In Five Languages",
    "Fly By Night",
    "Flying Blind",
    "Foiled At Every Turn",
    "Follow The Leader",
    "Follow The Herd",
    "Fool's Errand",
    "Foot The Bill",
    "Footloose And Fancy Free",
    "For Heaven's Sake",
    "For Pity's Sake",
    "For All Intents And Purposes",
    "For All The World To See",
    "For Better For Worse",
    "For Keeps",
    "For Richer For Poorer",
    "For The Love Of Mike",
    "For Your Information",
    "Fork It Over",
    "From Another Planet",
    "From Coast To Coast",
    "From Stem To Stern",
    "From This Day Forward",
    "Funny Money",
    "Game Set And Match",
    "Get A Grip On Yourself",
    "Get A Leg Up",
    "Get A Fix On It",
    "Get In Just Under The Wire",
    "Getting A Fair Shake",
    "Getting The Cold Shoulder",
    "Giant Size",
    "Give Him Enough Rope To Hang Himself",
    "Give Him His Walking Papers",
    "Give It The Old College Try",
    "Give It Your Best Effort",
    "Give Me A Sign",
    "Give Your Notice",
    "Glad To Meet You",
    "Go Ahead Sue Me",
    "Go Figure",
    "Go For Broke",
    "Go Out Of Your Way",
    "Going Full Throttle",
    "Going Going Gone",
    "Going Great Guns",
    "Going To Town",
    "Going By The Script",
    "Gone Fishin'",
    "Gone To Seed",
    "Good Grief",
    "Good For Nothing",
    "Graduate With Honors",
    "Great Day In The Morning",
    "Grow Up",
    "Guess Again",
    "Guilty As Charged",
    "Halt Who Goes There",
    "Hand Over Fist",
    "Hands Up",
    "Hang Around",
    "Hang On",
    "Hardly A Day Goes By",
    "Haste Makes Waste",
    "Have A Nice Day",
    "Haven't Seen Hide Nor Hair Of Him",
    "Having A Ball",
    "He Got A Raw Deal",
    "He Speaks Very Highly Of You",
    "He Who Hesitates Is Lost",
    "He's Got A Few Loose Screws",
    "Heaven And Earth",
    "Heaven Only Knows",
    "Heavens Above",
    "Here And Now",
    "High And Dry",
    "Highs And Lows",
    "His And Hers",
    "Hit The Jackpot",
    "Hold On Tight",
    "Holding Up Your End Of The Bargain",
    "Home Sweet Home",
    "How Much Do You Need",
    "Hurry Home",
    "I Beg Your Pardon",
    "I Can Manage Just Fine",
    "I Can't Recall",
    "I Demand My Rights",
    "I Don't Mind If I Do",
    "I Don't Want To Burst Your Bubble",
    "I Gave At The Office",
    "I Gave It Up For Lent",
    "I Hate To Eat And Run",
    "I Have No Use For It",
    "I Have Nothing More To Say",
    "I Have It All At My Fingertips",
    "I Haven't Got A Clue",
    "I Hear You Loud And Clear",
    "I Knew It All Along",
    "I Know What I Saw",
    "I Like It Like That",
    "I Mean What I Say",
    "I Regret To Inform You",
    "I Second The Motion",
    "I Spoke Too Soon",
    "I Tip My Hat To You",
    "I Want To Be Alone",
    "I Wash My Hands Of It",
    "I'll Be With You In A Minute",
    "I'll Run A Tab",
    "I'll Take You At Your Word",
    "I'm Looking Forward To It",
    "I'm Fond Of You",
    "I'm In Seventh Heaven",
    "I'm Not Buying It",
    "I'm Out Of Here",
    "I'm Stuck On You",
    "I'm Trying To Get The Kinks Out",
    "I've Formed An Opinion",
    "I've Got My Money On You",
    "If Looks Could Kill",
    "In Broad Daylight",
    "In Fits And Starts",
    "In Hot Water",
    "In My Neck Of The Woods",
    "In One Ear And Out The Other",
    "In The Face Of Danger",
    "In The Flesh",
    "In The Line Of Duty",
    "In The Nick Of Time",
    "In The Prime Of Life",
    "In Your Neck Of The Woods",
    "In Your Stocking Feet",
    "It Couldn't Happen To A Nicer Guy",
    "It Didn't Pan Out",
    "It Didn't Ring True",
    "It Never Fails",
    "It Seems Like Only Yesterday",
    "It Takes All Kinds",
    "It Takes Guts",
    "It Takes Two To Tango",
    "It Varies From Day To Day",
    "It Will Come To Pass",
    "It's A Crying Shame",
    "It's A Dog-Eat-Dog World",
    "It's A Living",
    "It's A Pity",
    "It's A Snap",
    "It's All The Rage",
    "It's High Time",
    "It's No Skin Off My Nose",
    "It's No Use Crying Over Spilt Milk",
    "It's Not All It's Cracked Up To Be",
    "It's Only Money",
    "It's Your Move",
    "Jumping To Conclusions",
    "Just Like Starting Over",
    "Just One Of The Guys",
    "Keep At It",
    "Keep It To Yourself",
    "Keep It Under Wraps",
    "Keep The Peace",
    "Keep Those Cards And Letters Coming",
    "Keep Time",
    "Keep Your Eyes Peeled",
    "Keep Your Fingers Crossed",
    "Keep Your Head",
    "Keep Your Losses To A Minimum",
    "Keeping Things On An Even Keel",
    "Keeping Up With The Joneses",
    "Knock On Wood",
    "Land On Your Feet",
    "Landed With A Thud",
    "Larger Than Life",
    "Last But Not Least",
    "Lean And Mean",
    "Less Is More",
    "Let It Ride",
    "Let The Buyer Beware",
    "Let Your Hair Down",
    "Let's Make Tracks",
    "Let's Play Fair",
    "Lie In State",
    "Like A Bull In A China Shop",
    "Like Father Like Son",
    "Like Taking Candy From A Baby",
    "Like Two Peas In A Pod",
    "Listen To Reason",
    "Living Proof",
    "Living Up To Expectations",
    "Living In The Lap Of Luxury",
    "Lo And Behold",
    "Loads Of Fun",
    "Look Before You Leap",
    "Looks Aren't Everything",
    "Lost And Found",
    "Made In America",
    "Make Yourself Scarce",
    "Make A Funny Face",
    "Make Me An Offer I Can't Resist",
    "Make Time",
    "Make Way",
    "Making A Big Splash",
    "Making Eye Contact",
    "Meeting On Neutral Ground",
    "Might Makes Right",
    "Mind Your Own Business",
    "Moving At A Snail's Pace",
    "My Suitcases Are Packed",
    "Neat As A Pin",
    "Never A Dull Moment",
    "Never Trust Anyone Over Thirty",
    "New And Improved",
    "News At Eleven",
    "Nice To See You",
    "No Offense Meant",
    "No Questions Asked",
    "No More Mr. Nice Guy",
    "No Place To Hide",
    "No Pun Intended",
    "No Room For Error",
    "No Takers",
    "Not At The Moment",
    "Not For All The Tea In China",
    "Not For Publication",
    "Not My Style",
    "Not On Your Life",
    "Not Worth A Hill Of Beans",
    "Nothing But Trouble",
    "Nothing But The Best",
    "Nothing In Common",
    "Nothing To Lose",
    "Now Hear This",
    "Nurse Your Wounds",
    "Odd Man Out",
    "Off Limits",
    "Off And Running",
    "Off To The Races",
    "Old Habits Die Hard",
    "On The Face Of It",
    "On The Launching Pad",
    "On The Level",
    "On The Make",
    "On The Right Track",
    "On The Straight And Narrow",
    "On The Tip Of My Tongue",
    "On The Verge Of Tears",
    "On The Move",
    "On The Run",
    "Once And For All",
    "Once In A Blue Moon",
    "One In A Million",
    "One Size Fits All",
    "Or So He Said",
    "Out Of Harm's Way",
    "Out Of Sync",
    "Out Of The Clear Blue Sky",
    "Out Of The Loop",
    "Out Of The Mouths Of Babes",
    "Out Of The Running",
    "Out Of Touch",
    "Out On My Own",
    "Over Hill And Dale",
    "Over The Edge",
    "Over The Top",
    "Over The Top",
    "Pacing Back And Forth",
    "Paid In Full",
    "Pamper Yourself",
    "Pancakes",
    "Paper Or Plastic",
    "Pay The Piper",
    "Pay Your Dues",
    "Pay Your Own Way",
    "People Will Talk",
    "Pick A Card Any Card",
    "Pick Up The Pace",
    "Pinching Pennies",
    "Pitch Black",
    "Places Everyone",
    "Places Everyone",
    "Playing By The Rules",
    "Please Wait Your Turn",
    "Poetic Justice",
    "Poetry In Motion",
    "Point Of No Return",
    "Power To The People",
    "Practice Makes Perfect",
    "Pretty As A Picture",
    "Pull Back On The Reins",
    "Pure Coincidence",
    "Pure As The Driven Snow",
    "Put Out The Fire",
    "Put Up A Good Fight",
    "Put Your Cards On The Table",
    "Putting Down Roots",
    "Quick As A Wink",
    "Quit While Yo're Ahead",
    "Ragged Around The Edges",
    "Rags To Riches Story",
    "Raining Cats And Dogs",
    "Ran Into A Little Snag",
    "Ranting And Raving",
    "Reaching The Outer Limits",
    "Recommended Dosage",
    "Related On My Mother's Side",
    "Remember When",
    "Ride Herd",
    "Riding Tall In The Saddle",
    "Right Of Way",
    "Ring Around The Rosie",
    "Rinky-Dink",
    "Roll The Dice",
    "Roll Up Your Sleeves",
    "Room For One More",
    "Rough Sailing Ahead",
    "Round Off To The Nearest Number",
    "Ruffled Feathers",
    "Run For Your Life",
    "Run Of The Mill",
    "Safe And Sound",
    "Sail Away",
    "Save Your Pennies For A Rainy Day",
    "Say Uncle",
    "Scared Stiff",
    "Season's Greetings",
    "Seize The Moment",
    "Selling Like Hotcakes",
    "Sent Him Packing",
    "Served On A Silver Platter",
    "Set Aside Some Time",
    "Set In His Ways",
    "Share And Share Alike",
    "Ship Ahoy",
    "Short-Changed",
    "Sight Unseen",
    "Sign Of The Times",
    "Signed Sealed And Delivered",
    "Sincerely Yours",
    "Sit Back And Relax",
    "Skating On Thin Ice",
    "Skin Deep",
    "Slim Pickings",
    "Slip Of The Tongue",
    "Smooth As Silk",
    "Snake Eyes",
    "Snuggle Up",
    "Sole Support",
    "Something To That Effect",
    "Soon Enough",
    "Sound The Wrong Note",
    "Spare The Rod And Spoil The Child",
    "Speak Of The Devil",
    "Spring Fever",
    "Stand Back And Watch The Fur Fly",
    "Stand Pat",
    "State Of The Art",
    "Stay On Your Toes",
    "Stay Tuned",
    "Steal Away",
    "Stem The Tide",
    "Stick It Out",
    "Stick Out Your Tongue",
    "Stir It Up",
    "Stop At Nothing",
    "Stop Making A Scene",
    "Store This End Up",
    "Stow Your Gear Here",
    "Street Smart",
    "Strictly Business",
    "Strike While The Iron Is Hot",
    "Strike A Pose",
    "Strike It Rich",
    "Supply And Demand",
    "Swan Song",
    "Sweep Under The Rug",
    "Sweet Dreams",
    "Swing Shift",
    "Take A Memo",
    "Take After Meals",
    "Take Detailed Notes",
    "Take Heart",
    "Take Your Pick",
    "Take A Back Seat",
    "Take A Load Off",
    "Take A Peek",
    "Take A Siesta",
    "Take It Easy",
    "Take It On The Chin",
    "Take It With A Grain Of Salt",
    "Taken By Storm",
    "Taking Advantage Of The Situation",
    "Taking Care Of Business",
    "Taking The Lion's Share",
    "Taking Turns",
    "Taking A Nap",
    "Talk A Blue Streak",
    "Talk It Over",
    "Thank Your Lucky Stars",
    "That Really Hits Home",
    "That Remains To Be Seen",
    "That Hits The Spot",
    "That Was Then This Is Now",
    "That'll Do The Trick",
    "That's Strong Medicine",
    "That's Tooting Your Own Horn",
    "That's All She Wrote",
    "That's All Well And Good",
    "That's Not The Worst Of It",
    "That's The Way Of The World",
    "The Bigger They Are The Harder They Fall",
    "The Calm Before The Storm",
    "The Check Is In The Mail",
    "The Crux Of The Matter",
    "The Day Before Yesterday",
    "The Early Bird Catches The Worm",
    "The Facts Of Life",
    "The Hustle And Bustle Of The Big City",
    "The Lay Of The Land",
    "The Letter Of The Law",
    "The Manner To Which You Are Accustomed",
    "The Milk Of Human Kindness",
    "The More The Merrier",
    "The Pen Is Mightier Than The Sword",
    "The Plot Thickens",
    "The Pot Calling The Kettle Black",
    "The Proof Is In The Pudding",
    "The Sky's The Limit",
    "The Talk Of The Town",
    "The Thrill Is Gone",
    "The Voice Of Reason",
    "The Whole Kit And Caboodle",
    "The Whole Truth And Nothing But The Truth",
    "The Worst Is Over",
    "There's No Time Like The Present",
    "There's No Such Thing As A Free Lunch",
    "Thick As Thieves",
    "Things Are Looking Up",
    "Things Are Tough All Over",
    "Things Change",
    "Third Time's A Charm",
    "Throw In The Towel",
    "Tighten The Purse Strings",
    "Time And A Half",
    "Timing Is Everything",
    "Tip The Scales",
    "To Say The Least",
    "To Your Health",
    "Together In Unison",
    "Too Many Cooks Spoil The Broth",
    "Torn In Half",
    "Transfer Of Deed",
    "Trial And Error",
    "True Blue",
    "Tune It Out",
    "Turned It Down Flat",
    "Turned Tail And Ran",
    "Twenty-Twenty Hindsight",
    "Twice As Nice",
    "Two Of A Kind",
    "Two Wrongs Don't Make A Right",
    "Until Further Notice",
    "Up To The Minute",
    "User-Friendly",
    "Vice Versa",
    "Wait Till Your Father Gets Home",
    "Walk Down The Aisle",
    "Walk The Plank",
    "Wandering Around In A Complete Fog",
    "Watch Out",
    "We're Going Places",
    "We're Poles Apart",
    "We're In The Money",
    "Westward Ho",
    "What A Good Idea",
    "What Do You Make Of That",
    "What Exactly Are You Trying To Say",
    "What's Mine Is Yours",
    "What's The Score",
    "What's Done Is Done",
    "When I Say Jump You Ask How High",
    "When Push Comes To Shove",
    "Where There's A Will There's A Way",
    "Where There's Smoke There's Fire",
    "Wherever You May Roam",
    "Whet Your Appetite",
    "Why Not",
    "Why The Long Face",
    "Win By A Nose",
    "Win Place Or Show",
    "Wipe Out",
    "Wishes Do Come True",
    "With A Little Luck",
    "Within Reason",
    "Without A Trace",
    "Woe Is Me",
    "Won't You Please Join Us",
    "Working Nine To Five",
    "Worth His Weight In Gold",
    "Write It Off",
    "Year In Year Out",
    "You Are Cordially Invited",
    "You Can Take My Place",
    "You Can't Have Your Cake And Eat It Too",
    "You Can't Make A Silk Purse Out Of A Sow's Ear",
    "You Can't Teach An Old Dog New Tricks",
    "Yo'll Have Your Say",
    "Yo're A Saint",
    "Yo're Not My Cup Of Tea",
    "Young At Heart",
    "Your Number's Up",
    "Your Place Or Mine",
    "Yours Truly"
],
"Event": [
    "A Face-Off In Hockey",
    "A Game Of Canasta",
    "A Game Of Simon Says",
    "A Schoolboy Prank",
    "Baby's First Steps",
    "Baseball Double-Header",
    "Birthday",
    "Breaking The Sound Barrier",
    "Bridge Game",
    "Buying On Margin",
    "Car Chase",
    "Charity Auction",
    "Concert",
    "Cramming For An Exam",
    "Cross-Country Skiing",
    "Curtain Call",
    "Demonstration",
    "Dinner",
    "Dire Emergency",
    "Double Feature",
    "Driver Education Class",
    "Earthquake",
    "Easter Egg Hunt",
    "Election",
    "Equipment Shutdown",
    "Festival",
    "Fire Drill",
    "Fistfight",
    "Flashback",
    "Flood",
    "Fly-Fishing",
    "Four-Car Pileup",
    "Four-Minute Mile",
    "Fourth Of July Parade",
    "Fund-Raiser",
    "Fund-Raising Party",
    "Getting A Speeding Ticket",
    "Graduation Day",
    "Grand Slam Home Run",
    "Heart-To-Heart Talk",
    "Hurricane",
    "Indian Summer",
    "Interview",
    "Investing For Your Retirement",
    "Jewish Holiday",
    "Joyride",
    "June Wedding",
    "Landslide",
    "Landslide Victory",
    "Lazy Afternoon",
    "Limbo Contest",
    "Locking Your Keys In Your Car",
    "Macy's Thanksgiving Day Parade",
    "Magic Show",
    "Mardi Gras",
    "Mass Protest",
    "Mowing The Lawn",
    "Nightmare",
    "Opening Night",
    "Overnight Trip",
    "Pajama Party",
    "Papal Audience",
    "Passing Inspection",
    "Passing Your Driver's Test",
    "Pep Rally",
    "Pilgrimage",
    "Pilgrimage",
    "Pony Ride",
    "Potluck Supper",
    "Quilting Bee",
    "Raffle",
    "Rainstorm",
    "Red-Letter Day",
    "Rehearsal",
    "Relay Race",
    "Repeat Performance",
    "Rite Of Passage",
    "Roller Skating",
    "Round Robin Tennis Tournament",
    "Sacking The Quarterback",
    "Scavenger Hunt",
    "Sending Someone A Fax",
    "Shadowboxing",
    "Shipwreck",
    "Shore Leave",
    "Showdown",
    "Snowstorm",
    "Soapbox Derby Race",
    "Sock Hop",
    "Solo Flight",
    "Space Shuttle Launch",
    "Stag Party",
    "State Fair",
    "State Of The Union Address",
    "Sudden Stop",
    "Swap Meet",
    "Sweepstakes Drawing",
    "Symposium",
    "Tag Sale",
    "Tailgate Party",
    "Taking A Nap",
    "Talent Search",
    "The Battle Of Hastings",
    "The Boston Tea Party",
    "The Centennial Olympics",
    "The Johnstown Flood",
    "The Olympic Long Jump",
    "The Olympic Pole Vault",
    "The Watergate Scandal",
    "The World Series",
    "The Battle Of The Bulge",
    "The Boston Tea Party",
    "The Cold War",
    "The War Of The Roses",
    "Touchdown",
    "Tournament",
    "Tournament Of Champions",
    "Tournament Of Roses Parade",
    "Track And Field Competition",
    "Trial By Jury",
    "Tropical Storm",
    "Under House Arrest",
    "Vacation",
    "Volleyball Game",
    "Water-Skiing",
    "Wedding",
    "Wild Goose Chase",
    "Wildcat Strike",
    "Yacht Race",
    "Yalta Conference",
    "Yearly Check-Up"
],
"Occupation": [
    "Ambassador",
    "Anthropologist",
    "Architect",
    "Astronaut",
    "Author",
    "Automobile Mechanic",
    "Bellman",
    "Blacksmith",
    "Bodyguard",
    "Bookkeeper",
    "Bus Driver",
    "Cab Driver",
    "Camp Counselor",
    "Carpenter",
    "Certified Public Accountant",
    "Checkout Clerk",
    "Chemist",
    "Circus Clown",
    "Copywriter",
    "Crossing Guard",
    "Custodian",
    "Customs Officer",
    "Dean Of Students",
    "Detective",
    "Doctor",
    "Doorman",
    "Editor",
    "Electrician",
    "Engineer",
    "English Teacher",
    "Executive Chef",
    "File Clerk",
    "Financial Planner",
    "Fire Fighter",
    "Fire Marshal",
    "Fireman",
    "Flight Attendant",
    "Football Coach",
    "Freelance Writer",
    "Front Desk Clerk",
    "General Contractor",
    "Hairdresser",
    "Handyman",
    "Headhunter",
    "High School Football Coach",
    "High School Guidance Counselor",
    "Insurance Underwriter",
    "Journalist",
    "Landlord",
    "Landscape Architect",
    "Lawyer",
    "Lifeguard",
    "Longshoreman",
    "Magazine Editor",
    "Masseur",
    "Mathematician",
    "Mayor",
    "Method Actor",
    "Music Teacher",
    "Newspaper Columnist",
    "Night Manager",
    "Obstetrician",
    "Office Manager",
    "Parking Attendant",
    "Pharmacist",
    "Pit Boss",
    "Plain Clothes Detective",
    "Playwright",
    "Plumber",
    "Podiatrist",
    "Police Lieutenant",
    "Police Officer",
    "Portrait Painter",
    "Postmaster General",
    "Preacher",
    "Real Estate Salesman",
    "Repairman",
    "Reporter",
    "Research Associate",
    "Sailor",
    "Sales Clerk",
    "School Principal",
    "School Teacher",
    "Scribe",
    "Secretary",
    "Security Guard",
    "Senate Page",
    "Shoemaker",
    "Short-Order Cook",
    "Stockbroker",
    "Tailor",
    "Talk Show Host",
    "Telephone Operator",
    "Telephone Repairman",
    "Test Pilot",
    "Tour Guide",
    "Truck Driver",
    "Underwriter",
    "University Professor",
    "Used Car Salesman",
    "Usher",
    "Violinist",
    "Waiter",
    "Waitress",
    "Writer"
]
}